# Ivy Plus MARC Analysis

This notebook processes MARC data from Ivy Plus libraries to identify unique items held by Penn that are not held by other institutions in the consortium.

## Enhanced Normalization and Matching

The matching process has been improved with multiple levels of matching and enhanced field extraction:

### 1. **Multi-Level Matching Strategy**
   - **Strict Match Keys**: Precise title, edition, and year matching
   - **Fuzzy Match Keys**: Broader matching with aggressive normalization for catching variations

### 2. **Enhanced Identifier Extraction**
   - **OCLC Numbers**: Handles all variants (ocm, ocn, on prefixes) and leading zeros
   - **ISBN Core**: Extracts the core ISBN for matching different formats of the same work
   - **Publication Year**: Now checks both F260 and F264 fields (many modern records use F264)
   - **LCCN**: Standardized to handle different formats and prefixes

### 3. **Special Handling**
   - **Multi-Volume Detection**: Identifies and properly handles multi-volume sets to prevent false positives
   - **Smart Title Normalization**: Preserves important distinctions while removing true noise

### 4. **Match Key Validation**
   - Each match key is validated for quality to detect potential issues
   - Short or generic match keys are flagged
   - Match key quality metrics are saved for analysis
   - Distribution statistics for different match types

### 5. **Field Selection**
   - Leader (FLDR) is included for record type identification
   - Core bibliographic fields (F001, F010, F020, F245, F250, F260, F264, F035) are used
   - F264 added for modern publication data
   - F035 for OCLC number extraction

This approach provides:
-  multiple match levels
-  enhanced OCLC and ISBN extraction
-  multi-volume detection
-  improved modern record support with F264 field processing

## Initial Load - Institution-specific Processing
Converts MARC to Parquet format for faster processing, maintaining institution-specific separation. This step ensures that each institution's MARC files are converted to separate Parquet files for consistent downstream processing.

The conversion includes the leader field (FLDR) for each record. The leader contains important information about the record structure, material type, and bibliographic level.

## Subsequent Runs
If you have already run the notebook, you may rely on caches and append for new data if available and necessary.


## HIGH MEMORY REQUIREMENT

**This notebook is configured for a high-RAM server environment with the following specifications:**

- **260GB driver memory allocation** (requires ~300GB total system RAM)
- **12 cores** for parallel processing
- Optimized for a **Linode 300GB server**

In [2]:
import sys, os
print("Python executable:", sys.executable)
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
print("PYSPARK_PYTHON:", os.environ["PYSPARK_PYTHON"])
print("PYSPARK_DRIVER_PYTHON:", os.environ["PYSPARK_DRIVER_PYTHON"])
!{sys.executable} -m pip install pymarc poetry fuzzywuzzy python-Levenshtein langdetect pyspark marctable==0.6.0

In [3]:
# Define paths for your PySpark server
# Update these paths to match your server's directory structure
input_dir = "/home/jovyan/work/July-2025-PODParquet/initial_parquet"  # Where your parquet files are located
output_dir = "/home/jovyan/work/July-2025-PODParquet/pod-processing-outputs"  # Where to save the results

# Create output directory if it doesn't exist
import os
os.makedirs(output_dir, exist_ok=True)

print(f"Input directory: {input_dir}")
print(f"Output directory: {output_dir}")

In [4]:
import os
import time
from pyspark.sql import SparkSession

# Clean up any existing Spark sessions
try:
    if 'spark' in globals():
        spark.stop()
        time.sleep(2)  # Give it time to clean up
except:
    pass

# Clear environment variables that might conflict
for key in list(os.environ.keys()):
    if 'SPARK' in key or 'JAVA' in key or 'PYSPARK' in key:
        del os.environ[key]

# Set JAVA_HOME explicitly
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'

# Create temp directory
os.makedirs('/tmp/spark-temp', exist_ok=True)

# Create Spark session with all configurations at once
# Since we know 200GB works from your test, we'll use that
print("Creating Spark session with full configuration...")

spark = SparkSession.builder \
    .appName("PodProcessing-Stable") \
    .master("local[12]") \
    .config("spark.driver.memory", "260g") \
    .config("spark.driver.maxResultSize", "200g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.adaptive.skewJoin.enabled", "true") \
    .config("spark.sql.adaptive.localShuffleReader.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "400") \
    .config("spark.memory.fraction", "0.6") \
    .config("spark.memory.storageFraction", "0.3") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.execution.arrow.maxRecordsPerBatch", "10000") \
    .config("spark.sql.parquet.enableVectorizedReader", "true") \
    .config("spark.sql.parquet.columnarReaderBatchSize", "2048") \
    .config("spark.sql.autoBroadcastJoinThreshold", "30m") \
    .config("spark.cleaner.periodicGC.interval", "5min") \
    .config("spark.cleaner.referenceTracking.cleanCheckpoints", "true") \
    .config("spark.local.dir", "/tmp/spark-temp") \
    .config("spark.sql.files.maxPartitionBytes", "134217728") \
    .config("spark.sql.files.openCostInBytes", "4194304") \
    .config("spark.driver.memoryOverhead", "20g") \
    .config("spark.kryoserializer.buffer.max", "1024m") \
    .config("spark.rpc.message.maxSize", "256") \
    .config("spark.network.timeout", "300s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .config("spark.rdd.compress", "true") \
    .config("spark.pyspark.python", sys.executable) \
    .config("spark.pyspark.driver.python", sys.executable) \
    .getOrCreate()

print("✅ Spark session initialized with 200GB memory and optimized settings!")
print(f"Spark UI available at: {spark.sparkContext.uiWebUrl}")

# Test it works
print("\nTesting Spark with a simple operation...")
test_df = spark.range(100).selectExpr("id", "id * 2 as doubled")
test_df.show(5)

# Verify key configurations
print("\n📋 Key configurations:")
print(f"  - Driver memory: {spark.conf.get('spark.driver.memory')}")
print(f"  - Max result size: {spark.conf.get('spark.driver.maxResultSize')}")
print(f"  - Memory fraction: {spark.conf.get('spark.memory.fraction')}")
print(f"  - Shuffle partitions: {spark.conf.get('spark.sql.shuffle.partitions')}")

print("\n✅ Spark session ready for processing!")

In [5]:
spark.catalog.clearCache()

In [6]:
# Spark SQL Functions - ENHANCED VERSION 2.0

from pyspark.sql.types import ArrayType, StringType
import pyspark.sql.functions as F

# Helper function to handle fields that might be strings or arrays
def handle_field_as_string(col_name):
    """
    Safely extract string value whether the field is a string or array.
    This version handles mixed types properly.
    """
    return F.when(
        F.col(col_name).isNotNull(),
        F.when(
            F.size(F.col(col_name)) >= 0,
            F.col(col_name).getItem(0)
        ).otherwise(
            F.col(col_name)
        )
    ).cast("string")

def extract_oclc_number_enhanced(df):
    """
    ENHANCED: Extract OCLC numbers from F035 field with ALL common patterns
    Handles ocm, ocn, on prefixes and leading zeros
    """
    return df.withColumn("oclc_number",
        F.when(F.col("F035").isNotNull() & (F.size(F.col("F035")) > 0),
            F.regexp_extract(
                F.concat_ws(" ", F.col("F035")),
                "\\(OCoLC\\)(?:ocm|ocn|on)?0*([0-9]+)",  # Handles prefixes AND leading zeros
                1
            )
        )
    )

def extract_publication_year_enhanced(df):
    """
    NEW: Check BOTH F260 and F264 for publication year
    Many newer records use F264 instead of F260
    """
    return df.withColumn("pub_year",
        F.coalesce(
            # First try F260
            F.when(F.col("F260").isNotNull() & (F.size(F.col("F260")) > 0),
                F.regexp_extract(F.col("F260").getItem(0), "(1[5-9][0-9]{2}|20[0-9]{2})", 1)
            ),
            # Then try F264 if F260 doesn't exist or is empty
            F.when(F.col("F264").isNotNull() & (F.size(F.col("F264")) > 0),
                F.regexp_extract(F.col("F264").getItem(0), "(1[5-9][0-9]{2}|20[0-9]{2})", 1)
            )
        )
    ).withColumn("pub_decade",
        F.when(F.col("pub_year").isNotNull(),
            F.concat(F.substring(F.col("pub_year"), 1, 3), F.lit("0s"))
        )
    )

def identify_multivolume(df):
    """
    NEW: Detect multi-volume works for special handling
    Prevents false uniqueness for sets where libraries hold different volumes
    """
    return df.withColumn("is_multivolume",
        F.col("F245").rlike("(?i)(v\\.|vol\\.|volume|pt\\.|part|tome|band|book)\\s*[0-9IVX]") |
        F.col("F245").rlike("(?i)\\[?[0-9]+(st|nd|rd|th)\\s+(v\\.|vol|edition)")
    ).withColumn("base_title_for_multivolume",
        F.when(F.col("is_multivolume"),
            # Strip volume indicators for matching
            F.regexp_replace(
                F.col("F245"),
                "(?i)[,;:]?\\s*(v\\.|vol\\.|volume|pt\\.|part|book)\\s*[0-9IVX]+.*$",
                ""
            )
        ).otherwise(F.col("F245"))
    )

def normalize_isbn_enhanced(df):
    """
    ENHANCED: Better ISBN normalization with core extraction
    Handles both ISBN-10 and ISBN-13 for better work-level matching
    """
    return df.withColumn("normalized_isbn",
        # F020 is array
        F.when(F.col("F020").isNotNull() & (F.size(F.col("F020")) > 0),
            F.regexp_replace(
                F.regexp_extract(F.col("F020").getItem(0), "([0-9X-]+)", 1),
                "[^0-9X]", ""
            )
        )
    ).withColumn("isbn_core",
        # Extract the core ISBN (ignoring check digit and prefix)
        F.when(F.length(F.col("normalized_isbn")) == 10,
            F.substring(F.col("normalized_isbn"), 1, 9)  # ISBN-10 core
        ).when(F.length(F.col("normalized_isbn")) == 13,
            F.substring(F.col("normalized_isbn"), 4, 9)  # ISBN-13 core (skip 978/979 prefix)
        )
    )

def create_smart_title_key(df):
    """
    NEW: Smarter title normalization that preserves important distinctions
    Less aggressive than fuzzy matching but catches more variations
    """
    return df.withColumn("title_normalized",
        # Remove only truly noise elements, keep important structure
        F.regexp_replace(
            F.regexp_replace(
                F.lower(F.trim(F.col("F245"))),
                "^(the|a|an|le|la|los|las|el|die|der|das|den|det)\\s+", ""
            ),
            "[\\[\\]\\(\\)/]", ""  # Remove only brackets and slashes, keep colons/semicolons
        )
    ).withColumn("title_first_significant",
        # First 5 significant words for better matching
        F.array_join(
            F.slice(
                F.split(F.col("title_normalized"), "\\s+"),
                1, 5
            ),
            " "
        )
    )

def create_match_key_spark_improved(df):
    """
    IMPROVED: Create better match keys using enhanced functions
    """
    # Apply all the enhanced transformations first
    df = df.transform(extract_publication_year_enhanced)
    df = df.transform(identify_multivolume)
    df = df.transform(create_smart_title_key)
    
    return df.withColumn("match_key", 
        F.concat_ws("_",
            # Use base title for multivolume works
            F.when(F.col("is_multivolume"),
                F.regexp_replace(F.col("base_title_for_multivolume"), "[^a-z0-9\\s]", "")
            ).otherwise(
                F.regexp_replace(F.col("title_normalized"), "[^a-z0-9\\s]", "")
            ),
            
            # Normalize edition (F250 is array)
            F.when(F.col("F250").isNotNull() & (F.size(F.col("F250")) > 0),
                F.regexp_replace(
                    F.lower(F.col("F250").getItem(0)), 
                    "(\\d+)(?:st|nd|rd|th)?\\s*(?:ed|edition)", "$1 ed"
                )
            ).otherwise(""),
            
            # Use enhanced year extraction
            F.coalesce(F.col("pub_year"), F.lit(""))
        )
    )

# Redirect old function to enhanced version for backward compatibility
def extract_oclc_number(df):
    """
    Redirect to enhanced version
    """
    return extract_oclc_number_enhanced(df)

# Keep the original create_match_key_spark for backward compatibility
def create_match_key_spark(df):
    """
    Create match keys - now uses improved version
    """
    return create_match_key_spark_improved(df)

def create_fuzzy_match_key(df):
    """
    Create FUZZY match keys for broader matching (catches more duplicates)
    """
    return df.withColumn("fuzzy_match_key",
        F.concat_ws("_",
            # More aggressive title normalization - remove ALL non-alphanumeric
            F.when(F.col("F245").isNotNull(),
                F.regexp_replace(
                    F.regexp_replace(
                        F.lower(F.trim(F.col("F245"))),
                        "^(the|a|an|le|la|el|los|las|die|der|das|den|det)\\s+", ""
                    ),
                    "[^a-z0-9]", ""  # Remove ALL punctuation and spaces
                )
            ).otherwise(""),
            
            # Just extract edition number, ignore format
            F.when(F.col("F250").isNotNull() & (F.size(F.col("F250")) > 0),
                F.regexp_extract(F.col("F250").getItem(0), "(\\d+)", 1)
            ).otherwise(""),
            
            # Year range (decade) instead of exact year
            F.when(F.col("pub_year").isNotNull(),
                F.col("pub_decade")
            ).otherwise("")
        )
    )

def create_work_level_key(df):
    """
    Create work-level match key (title + author only)
    """
    return df.withColumn("work_key",
        F.concat_ws("_",
            # Normalized title only
            F.when(F.col("F245").isNotNull(),
                F.regexp_replace(
                    F.lower(F.col("F245")),
                    "[^a-z0-9]", ""
                )
            ).otherwise(""),
            
            # Add author if available (F100 for personal, F110 for corporate)
            F.when(F.col("F100").isNotNull(),
                F.regexp_replace(F.lower(F.col("F100")), "[^a-z]", "")
            ).when(F.col("F110").isNotNull(),
                F.regexp_replace(F.lower(F.col("F110")), "[^a-z]", "")
            ).otherwise("")
        )
    )

def normalize_isbn_for_matching(df):
    """
    Enhanced ISBN normalization - redirects to enhanced version
    """
    return normalize_isbn_enhanced(df)

def normalize_ids_spark(df):
    """
    ENHANCED: Normalize ISBN and LCCN using improved functions
    """
    return df.transform(normalize_isbn_enhanced) \
        .withColumn("normalized_lccn", 
            F.when(F.col("F010").isNotNull(),
                F.regexp_replace(
                    F.trim(F.col("F010")),
                    "[^a-zA-Z0-9-]", ""
                )
            )
        )

def add_id_list_spark_enhanced(df):
    """
    ENHANCED: Create comprehensive id_list including ISBN core
    FIXED: Use concat to properly combine arrays
    """
    return df.withColumn("id_list",
        F.array_remove(
            F.array_distinct(
                F.concat(
                    # Standard identifiers
                    F.when(F.col("normalized_isbn").isNotNull() & (F.col("normalized_isbn") != ""), 
                        F.array(F.col("normalized_isbn"))).otherwise(F.array()),
                    F.when(F.col("isbn_core").isNotNull() & (F.col("isbn_core") != ""), 
                        F.array(F.col("isbn_core"))).otherwise(F.array()),
                    F.when(F.col("normalized_lccn").isNotNull() & (F.col("normalized_lccn") != ""), 
                        F.array(F.col("normalized_lccn"))).otherwise(F.array()),
                    F.when(F.col("oclc_number").isNotNull() & (F.col("oclc_number") != ""), 
                        F.array(F.col("oclc_number"))).otherwise(F.array()),
                    # Match keys
                    F.when(F.col("match_key").isNotNull() & (F.col("match_key") != ""), 
                        F.array(F.col("match_key"))).otherwise(F.array()),
                    F.when(F.col("fuzzy_match_key").isNotNull() & (F.col("fuzzy_match_key") != ""), 
                        F.array(F.col("fuzzy_match_key"))).otherwise(F.array()),
                    F.when(F.col("work_key").isNotNull() & (F.col("work_key") != ""), 
                        F.array(F.col("work_key"))).otherwise(F.array())
                )
            ),
            ""  # Remove empty strings
        )
    )

def validate_match_key_spark(df):
    """
    Validate match keys using Spark SQL functions
    """
    return df.withColumn("is_valid_match_key",
        (F.length(F.col("match_key")) >= 5) &
        (~F.col("match_key").rlike("^(book|text|edition|volume|vol|publication|report)_\\d+$"))
    ).withColumn("match_key_message",
        F.when(F.length(F.col("match_key")) < 5, "Match key too short")
         .when(F.col("match_key").rlike("^(book|text|edition|volume|vol|publication|report)_\\d+$"), "Generic match key")
         .otherwise("Valid match key")
    )

def process_institution_optimized(df, institution_name):
    """
    ENHANCED: Apply all enhanced optimizations to an institution's DataFrame
    """
    return (df
        .withColumn("source", F.lit(institution_name))
        .transform(extract_oclc_number_enhanced)  # ENHANCED OCLC
        .transform(extract_publication_year_enhanced)  # NEW: F264 support
        .transform(identify_multivolume)  # NEW: Multi-volume detection
        .transform(normalize_ids_spark)   # Enhanced with ISBN core
        .transform(create_match_key_spark_improved)  # IMPROVED match key
        .transform(create_fuzzy_match_key)  # Keep existing fuzzy
        .transform(create_work_level_key)   # Keep existing work-level
        .transform(add_id_list_spark_enhanced)  # Enhanced with ISBN core
        .transform(validate_match_key_spark)
    )

print("✅ ENHANCED Spark SQL functions loaded - VERSION 2.0")
print("✅ Major improvements:")
print("  - OCLC extraction handles all variants (ocm, ocn, on prefixes + leading zeros)")
print("  - Publication year checks both F260 and F264")
print("  - Multi-volume work detection prevents false positives")
print("  - ISBN core extraction for better work-level matching")
print("  - Smarter title normalization preserves important distinctions")
print("  - Backward compatible with existing code")
print("✅ FIXED: id_list generation now properly uses F.concat() to combine arrays")

# Institution-Specific MARC to Parquet Conversion Functions


In [ ]:
# Institution-Specific MARC to Parquet Conversion Functions

import os
import tempfile
import glob
import logging
from typing import Optional, Dict, List, Tuple
import re
from pymarc import Record, MARCReader
import subprocess
import sys

# First, let's verify marctable installation
print("=== Checking marctable installation ===")
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")

# Check if marctable can be imported
try:
    import marctable
    print(f"✅ marctable module can be imported")
    print(f"   Module location: {marctable.__file__}")
except ImportError as e:
    print(f"❌ Cannot import marctable: {e}")

# Check if marctable command is available in PATH
try:
    result = subprocess.run(['which', 'marctable'], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✅ marctable command found at: {result.stdout.strip()}")
    else:
        print("❌ marctable command not found in PATH")
        
        # Try to find it in common locations
        possible_paths = [
            os.path.join(sys.prefix, 'bin', 'marctable'),
            os.path.join(os.path.expanduser('~'), '.local', 'bin', 'marctable'),
            '/usr/local/bin/marctable',
            '/opt/conda/bin/marctable'
        ]
        
        for path in possible_paths:
            if os.path.exists(path):
                print(f"   Found marctable at: {path}")
                # Add to PATH for this session
                os.environ['PATH'] = os.path.dirname(path) + ':' + os.environ.get('PATH', '')
                print(f"   Added {os.path.dirname(path)} to PATH")
                break
except Exception as e:
    print(f"Error checking for marctable: {e}")

# Setup logging for MARC conversion
log_dir = f'{output_dir}/logs'

os.makedirs(log_dir, exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(os.path.join(log_dir, 'marc2parquet.log')),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

def process_file_with_recovery(file: str, institution: str) -> bool:
    """Process a MARC file with maximum error recovery"""
    try:
        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
        
        # Create a temporary file for processing
        with tempfile.NamedTemporaryFile(delete=False) as temp:
            temp_file = temp.name
        
        # Create institution-specific output filename
        base = os.path.basename(file)
        # Remove _filtered from the output name to avoid double naming
        clean_base = base.replace('_filtered', '')
        output_file = os.path.join(output_dir, 
                           f"{institution}_{clean_base.replace('.mrc', '')}-marc21.parquet")
        
        # Process MARC file
        written_count, report = safe_read_marc_file_with_recovery(file, temp_file)
        
        # Proceed if we have at least some records
        if written_count == 0:
            error_msg = f"No records could be processed from {file}"
            logger.error(error_msg)
            print(f"ERROR: {error_msg}")
            return False
        
        # Check temp file exists and has content
        if not os.path.exists(temp_file):
            error_msg = f"Temporary file {temp_file} does not exist!"
            logger.error(error_msg)
            print(f"ERROR: {error_msg}")
            return False
            
        temp_file_size = os.path.getsize(temp_file)
        print(f"  - Temp file size: {temp_file_size:,} bytes")
        
        # Try different ways to run marctable
        marctable_cmd = None
        
        # Method 1: Try direct command
        if subprocess.run(['which', 'marctable'], capture_output=True).returncode == 0:
            marctable_cmd = ['marctable', 'parquet', temp_file, output_file]
        # Method 2: Try with python -m
        else:
            marctable_cmd = [sys.executable, '-m', 'marctable', 'parquet', temp_file, output_file]
        
        print(f"Running command: {' '.join(marctable_cmd)}")
        
        # Run marctable command with better error capture
        try:
            result = subprocess.run(
                marctable_cmd,
                capture_output=True, 
                text=True, 
                check=True
            )
            
            if result.stdout:
                print(f"  - stdout: {result.stdout}")
            if result.stderr:
                print(f"  - stderr: {result.stderr}")
                
            # Check if output file was created
            if os.path.exists(output_file):
                output_size = os.path.getsize(output_file)
                success_msg = f"SUCCESS: Created {output_file} ({output_size:,} bytes) with {written_count} {institution} records"
                logger.info(success_msg)
                print(success_msg)
                return True
            else:
                error_msg = f"Output file {output_file} was not created"
                logger.error(error_msg)
                print(f"ERROR: {error_msg}")
                return False
                
        except subprocess.CalledProcessError as e:
            error_msg = f"marctable command failed with exit code {e.returncode}"
            logger.error(error_msg)
            print(f"ERROR: {error_msg}")
            print(f"  - stdout: {e.stdout}")
            print(f"  - stderr: {e.stderr}")
            return False
        except FileNotFoundError:
            error_msg = "marctable command not found"
            logger.error(error_msg)
            print(f"ERROR: {error_msg}")
            return False
            
    except Exception as e:
        error_msg = f"Unexpected error processing {institution} file {file}: {str(e)}"
        logger.error(error_msg)
        print(f"ERROR: {error_msg}")
        import traceback
        traceback.print_exc()
        return False
        
    finally:
        if 'temp_file' in locals() and temp_file and os.path.exists(temp_file):
            try:
                os.remove(temp_file)
            except Exception as e:
                logger.error(f"Cleanup error for {temp_file}: {str(e)}")

def safe_read_marc_file_with_recovery(input_file: str, output_file: str) -> Tuple[int, Dict]:
    """
    Read MARC file with aggressive error recovery, 
    capturing as many records as possible
    """
    written_count = 0
    error_count = 0
    recovered_count = 0
    error_types = {}
    
    try:
        with open(input_file, 'rb') as fh:
            # Try to read the entire file into memory for better recovery
            file_content = fh.read()
            
        # First pass: try standard reading
        try:
            reader = MARCReader(file_content, to_unicode=True, force_utf8=True, 
                              hide_utf8_warnings=True, utf8_handling='ignore')
            
            with open(output_file, 'wb') as out:
                for record in reader:
                    if record is not None:
                        try:
                            out.write(record.as_marc())
                            written_count += 1
                        except Exception as e:
                            error_count += 1
                            error_type = type(e).__name__
                            error_types[error_type] = error_types.get(error_type, 0) + 1
                            
        except Exception as e:
            logger.warning(f"Standard reading failed: {str(e)}. Attempting recovery...")
            
            # Second pass: chunk-based recovery
            chunk_size = 2048  # Start with 2KB chunks
            position = 0
            
            with open(output_file, 'wb') as out:
                while position < len(file_content):
                    # Find next record start (0x1D)
                    start = file_content.find(b'\x1d', position)
                    if start == -1:
                        break
                    
                    # Find end of this record (next 0x1D or end of file)
                    end = file_content.find(b'\x1d', start + 1)
                    if end == -1:
                        end = len(file_content)
                    
                    # Try to parse this chunk as a record
                    try:
                        chunk = file_content[start:end + 1]
                        record = Record(data=chunk, to_unicode=True, force_utf8=True)
                        if record and hasattr(record, 'leader'):
                            out.write(record.as_marc())
                            written_count += 1
                            recovered_count += 1
                    except:
                        error_count += 1
                    
                    position = end + 1
                    
    except Exception as e:
        logger.error(f"Critical error reading file: {str(e)}")
        error_count += 1
        error_types['Critical'] = 1
    
    # Calculate success rate
    total_attempted = written_count + error_count
    success_rate = (written_count / total_attempted * 100) if total_attempted > 0 else 0
    
    report = {
        'written': written_count,
        'errors': error_count,
        'recovered': recovered_count,
        'success_rate': success_rate,
        'error_types': error_types
    }
    
    return written_count, report

def detect_encoding(file_path: str, sample_size: int = 10000) -> str:
    """
    Detect the encoding of a MARC file
    """
    try:
        with open(file_path, 'rb') as f:
            sample = f.read(sample_size)
        
        # First try to detect using chardet
        import chardet
        result = chardet.detect(sample)
        confidence = result.get('confidence', 0)
        
        if confidence > 0.7:
            return result['encoding']
        
        # Fallback: check for MARC-8 indicators
        if b'\x1b' in sample:  # ESC character often indicates MARC-8
            return 'MARC-8'
        
        # Default to UTF-8
        return 'utf-8'
    except:
        return 'utf-8'

def convert_marc_to_parquet_batch(marc_files: List[str], output_dir: str, 
                                 batch_name: str = "batch") -> List[str]:
    """
    Convert multiple MARC files to Parquet format in batch
    """
    success_files = []
    
    for i, marc_file in enumerate(marc_files):
        try:
            # Create output filename
            base_name = os.path.basename(marc_file).replace('.mrc', '')
            output_file = os.path.join(output_dir, f"{batch_name}_{base_name}.parquet")
            
            # Process with recovery
            logger.info(f"Processing {marc_file} ({i+1}/{len(marc_files)})")
            temp_file = f"/tmp/temp_{batch_name}_{i}.mrc"
            
            written, report = safe_read_marc_file_with_recovery(marc_file, temp_file)
            
            if written > 0:
                # Convert to parquet using marctable
                cmd = f"marctable parquet {temp_file} {output_file}"
                result = os.system(cmd)
                
                if result == 0:
                    success_files.append(output_file)
                    logger.info(f"Successfully converted {marc_file} -> {output_file}")
                else:
                    logger.error(f"marctable failed for {marc_file}")
                
                # Cleanup temp file
                if os.path.exists(temp_file):
                    os.remove(temp_file)
            else:
                logger.error(f"No records recovered from {marc_file}")
                
        except Exception as e:
            logger.error(f"Failed to process {marc_file}: {str(e)}")
            continue
    
    return success_files

def process_institution_marc_files(institution: str, files: List[str]) -> bool:
    """
    Process all MARC files for a single institution
    """
    print(f"\n{'='*60}")
    print(f"Processing {institution.upper()} - {len(files)} files")
    print(f"{'='*60}")
    
    all_success = True
    
    for file in files:
        success = process_file_with_recovery(file, institution)
        all_success = all_success and success
    
    return all_success

print("✅ All MARC processing functions loaded successfully")

In [ ]:
# Process MARC files to Parquet for each institution - BATCH PROCESSING VERSION
import glob
import os

# Update input_dir to point to your specific directory
input_dir = "/home/jovyan/work/July-2025-PODParquet/pod-processing-outputs/final"
print(f"Using input directory: {input_dir}")

print("=== PROCESSING MARC FILES TO PARQUET - BATCH MODE ===")
print("This will convert MARC files from each institution to Parquet format")
print("⚠️  WARNING: Source .mrc files will be DELETED immediately after conversion to save space!\n")

# Check initial disk space
import subprocess
df_output = subprocess.run("df -h /", shell=True, capture_output=True, text=True)
print("📊 Initial disk usage:")
print(df_output.stdout)

# Define institutions in order of file size (smallest first to maximize success)
# This order processes smaller files first to free up space gradually
institutions_by_size = [
    ('harvard', 110.0),
]

# Dictionary to track processing results
processing_results = {}
deleted_files = []
space_freed = 0
cumulative_space_freed = 0

# Process in batches
print("\n📋 Processing order (smallest to largest):")
for inst, size in institutions_by_size:
    print(f"  - {inst}: {size:.1f} GB")

print("\nStarting batch processing...\n")

for institution, expected_size_gb in institutions_by_size:
    print(f"\n{'='*60}")
    print(f"Processing {institution.upper()} (expected ~{expected_size_gb} GB)")
    print(f"{'='*60}")
    
    # Check current disk space
    df_result = subprocess.run("df -h / | tail -1", shell=True, capture_output=True, text=True)
    available_space = df_result.stdout.split()[3]
    print(f"📊 Current available space: {available_space}")
    
    # Look for the MARC file
    marc_file = f"{input_dir}/{institution}_filtered.mrc"
    
    if not os.path.exists(marc_file):
        print(f"⚠️  No MARC file found for {institution}")
        processing_results[institution] = {"status": "no_files", "count": 0}
        continue
    
    # Get the actual file size
    file_size = os.path.getsize(marc_file)
    print(f"Found MARC file: {os.path.basename(marc_file)} ({file_size/1024/1024/1024:.2f} GB)")
    
    # Check if we have enough space for conversion (need at least 50% of file size for safety)
    df_bytes_result = subprocess.run("df / | tail -1", shell=True, capture_output=True, text=True)
    available_bytes = int(df_bytes_result.stdout.split()[3]) * 1024  # Convert KB to bytes
    
    required_space = file_size * 0.5  # Need 50% of file size for conversion
    if available_bytes < required_space:
        print(f"⚠️  WARNING: May not have enough space!")
        print(f"  - Available: {available_bytes/1024/1024/1024:.2f} GB")
        print(f"  - Required: {required_space/1024/1024/1024:.2f} GB")
        # Non-interactive mode: proceed automatically
        print("🤖 Non-interactive mode: proceeding despite low space warning.")
        # If you prefer to skip instead of proceed, uncomment the next lines:
        # processing_results[institution] = {"status": "skipped_low_space", "count": 0}
        # print("Skipping this file due to low disk space.")
        # continue
    
    marc_files = [marc_file]
    
    # Process the institution's file
    try:
        success = process_institution_marc_files(institution, marc_files)
        
        if success:
            # Check what was created
            output_pattern = f"{output_dir}/{institution}_*marc21.parquet"
            created_files = glob.glob(output_pattern)
            
            if created_files:
                print(f"\n✅ Successfully created {len(created_files)} Parquet file(s) for {institution}")
                total_parquet_size = 0
                for cf in created_files:
                    parquet_size = os.path.getsize(cf)
                    total_parquet_size += parquet_size
                    print(f"  - {os.path.basename(cf)} ({parquet_size/1024/1024:.2f} MB)")
                
                # Show compression ratio
                compression_ratio = (1 - total_parquet_size/file_size) * 100
                print(f"  📊 Compression: {compression_ratio:.1f}% reduction")
                
                # DELETE THE SOURCE MARC FILE IMMEDIATELY
                try:
                    os.remove(marc_file)
                    deleted_files.append(marc_file)
                    space_freed = file_size
                    cumulative_space_freed += space_freed
                    print(f"  🗑️  Deleted source file: {os.path.basename(marc_file)}")
                    print(f"  💾 Freed: {space_freed/1024/1024/1024:.2f} GB")
                    print(f"  📊 Total freed so far: {cumulative_space_freed/1024/1024/1024:.2f} GB")
                except Exception as e:
                    print(f"  ⚠️  WARNING: Could not delete source file: {e}")
                
                processing_results[institution] = {
                    "status": "success",
                    "count": len(created_files),
                    "files": created_files,
                    "source_deleted": marc_file in deleted_files,
                    "space_freed_gb": space_freed/1024/1024/1024
                }
                
                # Show updated disk space
                df_after = subprocess.run("df -h / | tail -1", shell=True, capture_output=True, text=True)
                print(f"\n📊 Disk space after {institution}:")
                print(f"   {df_after.stdout}")
                
            else:
                print(f"⚠️  No Parquet files were created for {institution}")
                print(f"  💾 Keeping source file: {os.path.basename(marc_file)}")
                processing_results[institution] = {"status": "no_output", "count": 0}
        else:
            print(f"❌ Failed to process {institution} MARC files")
            print(f"  💾 Keeping source file due to error: {os.path.basename(marc_file)}")
            processing_results[institution] = {"status": "failed", "count": 0}
            
    except Exception as e:
        print(f"❌ Error processing {institution}: {str(e)}")
        print(f"  💾 Keeping source file due to error: {os.path.basename(marc_file)}")
        processing_results[institution] = {"status": "error", "count": 0, "error": str(e)}

# Final summary report
print("\n" + "="*60)
print("PROCESSING SUMMARY")
print("="*60)

successful = sum(1 for r in processing_results.values() if r["status"] == "success")
failed = sum(1 for r in processing_results.values() if r["status"] in ["failed", "error"])
skipped = sum(1 for r in processing_results.values() if r["status"] == "skipped")
no_files = sum(1 for r in processing_results.values() if r["status"] == "no_files")

print(f"Total institutions: {len(institutions_by_size)}")
print(f"✅ Successful: {successful}")
print(f"❌ Failed: {failed}")
print(f"⏭️  Skipped: {skipped}")
print(f"📭 No files: {no_files}")

print("\nDetailed results:")
for inst, result in sorted(processing_results.items()):
    status_emoji = "✅" if result["status"] == "success" else "❌" if result["status"] in ["failed", "error"] else "⏭️" if result["status"] == "skipped" else "📭"
    deleted_emoji = "🗑️" if result.get("source_deleted", False) else "💾"
    space_info = f" (freed {result.get('space_freed_gb', 0):.1f} GB)" if result.get('space_freed_gb') else ""
    print(f"{status_emoji} {inst}: {result['status']} {deleted_emoji}{space_info}")

# Space savings report
print(f"\n💾 SPACE SAVINGS:")
print(f"  - Deleted {len(deleted_files)} source files")
print(f"  - Total space freed: {cumulative_space_freed/1024/1024/1024:.2f} GB")

# Final disk check
print("\n📊 Final disk usage:")
df_final = subprocess.run("df -h /", shell=True, capture_output=True, text=True)
print(df_final.stdout)

# List all created Parquet files
print(f"\n=== All Parquet files in {output_dir} ===")
all_parquet = glob.glob(f"{output_dir}/*.parquet")
print(f"Total Parquet files: {len(all_parquet)}")
total_parquet_size = 0
for pf in sorted(all_parquet):
    file_size = os.path.getsize(pf)
    total_parquet_size += file_size
    print(f"  - {os.path.basename(pf)} ({file_size/1024/1024:.2f} MB)")

print(f"\nTotal Parquet storage: {total_parquet_size/1024/1024/1024:.2f} GB")
print(f"Original MARC size: ~167 GB")
print(f"Compression ratio: {(1 - total_parquet_size/(167*1024*1024*1024))*100:.1f}%")

print("\n✅ Batch MARC to Parquet conversion complete!")

# Main Processing with Memory-Optimized Approach


In [ ]:
# Run this before re-processing
spark.catalog.clearCache()

In [ ]:
# Main Processing - Memory-Optimized with Batch Processing
import glob
import os
from pyspark.sql import DataFrame
import pyspark.sql.functions as F

print("=== STARTING MAIN PROCESSING ===")
print("This will process all institution parquet files and create the exploded dataset\n")

# Get all institution parquet files
parquet_files = glob.glob(f"{input_dir}/*.parquet")
print(f"Found {len(parquet_files)} institution parquet files to process")

# Process each institution and save to temp directory
temp_output_dir = f"{output_dir}/temp_processed"
os.makedirs(temp_output_dir, exist_ok=True)

processed_institutions = []

for file_path in parquet_files:
    # Extract institution name from filename
    filename = os.path.basename(file_path)
    institution = filename.split('_')[0]
    
    print(f"\nProcessing {institution}...")
    
    try:
        # Read institution data
        df = spark.read.parquet(file_path)
        # Skip counting - just process
        # record_count = df.count()
        # print(f"  - Records: {record_count:,}")
        
        # Apply all enhanced processing
        processed_df = process_institution_optimized(df, institution)
        
        # Save processed data
        temp_path = f"{temp_output_dir}/{institution}_processed.parquet"
        processed_df.write.mode("overwrite").parquet(temp_path)
        
        processed_institutions.append((institution, temp_path))
        print(f"  ✅ Saved to {temp_path}")
        
        # Clear cache to free memory
        spark.catalog.clearCache()
        
    except Exception as e:
        print(f"  ❌ Error processing {institution}: {str(e)}")
        continue

print(f"\n✅ Processed {len(processed_institutions)} institutions")

# Now create the exploded dataset by reading all processed files
print("\n=== CREATING EXPLODED DATASET ===")
print("This creates a row for each identifier/key in each record...")

# Read all processed institution files
processed_paths = [path for _, path in processed_institutions]
all_df = spark.read.parquet(*processed_paths)

# Create exploded dataset with id_list as key_array
all_df_with_key_array = all_df.withColumn("key_array", F.col("id_list"))

# Explode the key_array to create one row per key
all_df_exploded = all_df_with_key_array.select(
    "F001", "source", "match_key", "is_valid_match_key",
    F.explode("key_array").alias("key")
).filter(F.col("key").isNotNull())

# Persist exploded dataset for downstream cells
exploded_path = f"{output_dir}/all_df_exploded.parquet"
all_df_exploded.write.mode("overwrite").parquet(exploded_path)
print(f"\n✅ Saved exploded dataset to: {exploded_path}")

# Cache for immediate use
all_df_exploded.cache()
print(f"📌 Dataset cached in memory for subsequent analysis")

print(f"\n💡 Ready for uniqueness analysis")

In [ ]:
# Penn Overlap Analysis: compute unique_penn and overlap summary
from pyspark.sql import functions as F
import os

print("=== PENN OVERLAP ANALYSIS ===")
exploded_path = f"{output_dir}/all_df_exploded.parquet"
all_df_exploded = spark.read.parquet(exploded_path)
print(f"Loaded exploded dataset: {exploded_path}")

# Classify identifier types based on value patterns
id_classified = all_df_exploded.withColumn(
    "id_type",
    F.when(F.col("key").rlike("^[0-9X]{10,13}$"), F.lit("ISBN"))
     .when(F.col("key").rlike("^[0-9]{8,}$"), F.lit("OCLC"))
     .when(F.col("key").contains("_"), F.lit("MatchKey"))
     .when(F.col("key").rlike("^[A-Za-z0-9-]{5,}$"), F.lit("LCCN"))
     .otherwise(F.lit("OTHER"))
)
standard_types = ["ISBN", "OCLC", "LCCN", "MatchKey"]
df_std = id_classified.filter(F.col("id_type").isin(standard_types))

# Compute cross-institution collisions on standard ids
id_counts = df_std.groupBy("key").agg(F.countDistinct("source").alias("id_count"))
multi_inst_ids = id_counts.filter(F.col("id_count") > 1).select("key").distinct()
print(f"Standard IDs shared across institutions: {multi_inst_ids.count():,}")

# Compute Penn records that have no standard-id collisions with any non-Penn source
penn_df = all_df_exploded.filter(F.col("source") == F.lit("penn"))
penn_std = penn_df.join(df_std.select("key").distinct(), on="key", how="left_semi")
penn_unique_by_std = penn_std.join(multi_inst_ids, on="key", how="left_anti").select("F001").distinct()

# Materialize unique_penn only if not already present
unique_penn_path = f"{output_dir}/unique_penn.parquet"
if not os.path.exists(unique_penn_path):
    penn_full = spark.read.parquet(f"{output_dir}/penn_penn-marc21.parquet")
    unique_penn = penn_full.join(penn_unique_by_std, on="F001", how="inner")
    unique_penn.write.mode("overwrite").parquet(unique_penn_path)
    print(f"✅ Saved unique_penn to: {unique_penn_path}")
else:
    print(f"ℹ️ unique_penn already exists at {unique_penn_path}; skipping write")

# Overlap analysis summary per id key (always write for diagnostics)
overlap_summary = df_std.groupBy("key").agg(
    F.collect_set("source").alias("sources"),
    F.count("*").alias("total_occurrences"),
    F.countDistinct("source").alias("inst_count")
)
penn_overlap = penn_df.join(overlap_summary, on="key", how="left")
penn_overlap_path = f"{output_dir}/penn_overlap_analysis.parquet"
penn_overlap.write.mode("overwrite").parquet(penn_overlap_path)
print(f"✅ Saved penn_overlap_analysis to: {penn_overlap_path}")

In [ ]:
spark.catalog.clearCache()

In [ ]:
# Conservative Uniqueness Filtering (revised) - SAFE VERSION
from pyspark.sql import functions as F
import os

print("=== CONSERVATIVE UNIQUENESS FILTERING (REVISED) ===")

# Check if September 13 outputs already exist
conservative_path = f"{output_dir}/conservative_unique_penn.parquet"
conservative_filtered_path = f"{output_dir}/conservative_unique_penn_filtered.parquet"

# Check what we have from September 13
for path in [conservative_path, conservative_filtered_path]:
    if os.path.exists(path):
        success_file = os.path.join(path, "_SUCCESS")
        if os.path.exists(success_file):
            print(f"✅ {os.path.basename(path)} exists from September 13 - DATA IS SAFE")
            # Get file count to verify
            import glob
            part_files = glob.glob(os.path.join(path, "part-*.parquet"))
            print(f"   Contains {len(part_files)} parquet part files")

# Ask if you want to reprocess or just verify
response = input("\nDo you want to REPROCESS (overwrite) or just VERIFY existing data? (reprocess/verify): ").lower()

if response == 'verify':
    print("\n✅ Verifying existing September 13 data...")
    
    # Quick verification without full count
    for path in [conservative_path, conservative_filtered_path]:
        if os.path.exists(path):
            df = spark.read.parquet(path)
            print(f"\n{os.path.basename(path)}:")
            print(f"  - Schema verified: {len(df.columns)} columns")
            print(f"  - Sample record check: {df.limit(1).count() == 1}")
            
            # Show schema
            print("  - Columns:", ', '.join(df.columns[:10]), "...")
    
    print("\n✅ September 13 data is intact and ready to use!")
    
elif response == 'reprocess':
    print("\nReprocessing from scratch...")
    
    # Your existing logic WITHOUT the counts
    exploded_path = f"{output_dir}/all_df_exploded.parquet"
    if os.path.exists(exploded_path):
        all_df_exploded = spark.read.parquet(exploded_path)
        print(f"Loaded exploded dataset from {exploded_path}")
    else:
        print("⚠️ Exploded dataset not found on disk. Need to recreate.")
        # You'll need to recreate all_df_exploded here
    
    # Rest of your processing logic...
    id_classified = all_df_exploded.withColumn(
        "id_type",
        F.when(F.col("key").rlike("^[0-9X]{10,13}$"), F.lit("ISBN"))
         .when(F.col("key").rlike("^[0-9]{8,}$"), F.lit("OCLC"))
         .when(F.col("key").contains("_"), F.lit("MatchKey"))
         .when(F.col("key").rlike("^[A-Za-z0-9-]{5,}$"), F.lit("LCCN"))
         .otherwise(F.lit("OTHER"))
    )
    
    # Continue with processing...
    standard_types = ["ISBN", "OCLC", "LCCN", "MatchKey"]
    std_ids = id_classified.filter(F.col("id_type").isin(standard_types))
    
    # Multi-institution collisions on standard ids
    key_inst_count = std_ids.groupBy("key").agg(F.countDistinct("source").alias("inst_count"))
    multi_inst_keys = key_inst_count.filter(F.col("inst_count") > 1).select("key").distinct()
    
    # Penn unique by standard ids
    penn_rows = id_classified.filter(F.col("source") == F.lit("penn"))
    penn_std = penn_rows.join(std_ids.select("key").distinct(), on="key", how="left_semi")
    penn_unique_keys = penn_std.join(multi_inst_keys, on="key", how="left_anti").select("F001").distinct()
    
    # Cache for one count only
    penn_unique_keys.cache()
    print(f"Penn records unique by standard ids: {penn_unique_keys.count():,}")
    penn_unique_keys.unpersist()
    
    # Rest of processing...
    penn_full = spark.read.parquet(f"{input_dir}/penn_penn-marc21.parquet")
    conservative_unique_penn = penn_full.join(penn_unique_keys, on="F001", how="inner")
    
    # Apply filters
    has_533 = (F.col("F533").isNotNull() & (F.size(F.col("F533")) > 0))
    conservative_unique_penn_no533 = conservative_unique_penn.filter(~has_533)
    
    # HSP filtering
    possible_hsp_paths = [
        os.path.join(output_dir, "hsp_removed_mmsid.txt"),
        os.path.join(os.getcwd(), "hsp_removed_mmsid.txt"),
        "hsp_removed_mmsid.txt",
    ]
    
    hsp_path_found = next((p for p in possible_hsp_paths if os.path.exists(p)), None)
    if hsp_path_found:
        hsp_mmsids = spark.createDataFrame(
            [(line.strip(),) for line in open(hsp_path_found, "r", encoding="utf-8").read().splitlines() if line.strip()],
            schema=["F001"]
        )
        conservative_filtered = conservative_unique_penn_no533.join(hsp_mmsids, on="F001", how="left_anti")
        print(f"Applied HSP exclusion filter from: {hsp_path_found}")
    else:
        conservative_filtered = conservative_unique_penn_no533
        print("HSP exclusion list not found; skipping HSP filter")
    
    # Write WITHOUT counts in the print statements
    print(f"\nWriting conservative_unique_penn to: {conservative_path}")
    conservative_unique_penn.write.mode("overwrite").parquet(conservative_path)
    print(f"✅ Write complete")
    
    print(f"\nWriting conservative_unique_penn_filtered to: {conservative_filtered_path}")  
    conservative_filtered.write.mode("overwrite").parquet(conservative_filtered_path)
    print(f"✅ Write complete")
    
    print("\n✅ Reprocessing complete! Use the verification option to check record counts.")
else:
    print("\n✅ No changes made. Your September 13 data remains intact.")

In [ ]:
spark.catalog.clearCache()

In [ ]:
# Conservative Uniqueness Analysis - OPTIMIZED VERSION
# Disable broadcast joins to prevent timeout errors with large datasets
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")
spark.conf.set("spark.sql.broadcastTimeout", "1200")  # 20 minutes (if broadcast is re-enabled)

from pyspark.sql.functions import col, size, array_contains, collect_set, count, when
import pyspark.sql.functions as F
import glob
import os

print("=== CONSERVATIVE UNIQUENESS ANALYSIS (OPTIMIZED) ===")
print("Analyzing POD dataset for unique Penn records")
print("Applying stricter criteria to identify truly unique records\n")
print("⚠️ This version avoids expensive count operations to prevent hangs\n")

# Check if we have the exploded dataset on disk
exploded_path = f"{output_dir}/all_df_exploded.parquet"
if not os.path.exists(exploded_path):
    print("❌ Exploded dataset not found. Please run the main processing cell first.")
    raise FileNotFoundError(f"Missing required file: {exploded_path}")

print(f"✓ Loading exploded POD dataset from: {exploded_path}")
all_df_exploded = spark.read.parquet(exploded_path)

# STEP 1: Classify identifier types based on key patterns (matching Penn Overlap Analysis)
print("\n[1/7] Classifying identifier types...")
id_classified = all_df_exploded.withColumn(
    "id_type",
    F.when(F.col("key").rlike("^[0-9X]{10,13}$"), F.lit("ISBN"))
     .when(F.col("key").rlike("^[0-9]{8,}$"), F.lit("OCLC"))
     .when(F.col("key").contains("_"), F.lit("MatchKey"))
     .when(F.col("key").rlike("^[A-Za-z0-9-]{5,}$"), F.lit("LCCN"))
     .otherwise(F.lit("OTHER"))
)

# STEP 2: Filter to Penn records with standard ID types
print("\n[2/7] Filtering to Penn POD records with standard IDs...")
standard_types = ['OCLC', 'LCCN', 'ISBN', 'MatchKey']
penn_rows = id_classified.filter(col("source") == "penn")
penn_std = penn_rows.filter(col("id_type").isin(standard_types))
penn_std_exploded_count = penn_std.count()
penn_std_unique = penn_std.select("F001").distinct()
penn_std_count = penn_std_unique.count()
print(f"  → {penn_std_exploded_count:,} Penn identifier rows with standard IDs (exploded)")
print(f"  → {penn_std_count:,} unique Penn POD records (F001s) with standard IDs")

# STEP 3: Group by key across ALL INSTITUTIONS to identify shared vs unique IDs
print("\n[3/7] Grouping by key across ALL institutions to detect sharing...")
print("  ⏳ This may take several minutes for large datasets...")

# CRITICAL: Group across ALL institutions (not just Penn) to detect sharing
all_std = id_classified.filter(col("id_type").isin(standard_types))
grouped_standard = (
    all_std
    .groupBy("key")
    .agg(
        collect_set("source").alias("sources"),
        collect_set("F001").alias("f001_values")
    )
)

# Mark if shared across institutions (size > 1)
grouped_standard = grouped_standard.withColumn(
    "is_shared", 
    size(col("sources")) > 1
)

print("  → Grouping complete")

# STEP 4: Join back to get sharing status for Penn records
print("\n[4/7] Joining sharing status back to Penn records...")
penn_with_standard_ids = penn_std.join(
    grouped_standard.select("key", "is_shared"),
    on="key",
    how="left"
)

# STEP 5: Filter to unique Penn records (not shared)
print("\n[5/7] Filtering to unique Penn POD records (not shared with other institutions)...")
penn_unique_keys = penn_with_standard_ids.filter(col("is_shared") == False)

# Get distinct F001s from unique records
conservative_unique_f001s = penn_unique_keys.select("F001").distinct()
unique_count = conservative_unique_f001s.count()
print(f"  → {unique_count:,} unique Penn POD F001s identified (truly unique to Penn)")

# STEP 6: Join back to full Penn dataset to get complete records
print("\n[6/7] Retrieving full records for unique F001s...")
penn_full = all_df_exploded.filter(col("source") == "penn")
conservative_unique_penn = penn_full.join(
    conservative_unique_f001s,
    on="F001",
    how="inner"
)

final_count = conservative_unique_penn.select("F001").distinct().count()
print(f"  → {final_count:,} unique Penn POD records retrieved (verification)")

# STEP 7: Save results
print("\n[7/7] Saving results...")
conservative_unique_path = f"{output_dir}/conservative_unique_penn.parquet"
conservative_unique_penn.write.mode("overwrite").parquet(conservative_unique_path)
print(f"  ✓ Saved to: {conservative_unique_path}")

# Save just the F001 list for quick reference
f001_list_path = f"{output_dir}/conservative_unique_f001s.parquet"
conservative_unique_f001s.write.mode("overwrite").parquet(f001_list_path)
print(f"  ✓ F001 list saved to: {f001_list_path}")

# Summary statistics
print("\n" + "="*60)
print("CONSERVATIVE UNIQUENESS SUMMARY (POD DATA)")
print("="*60)
print(f"Penn POD records with standard IDs:   {penn_std_count:,}")
print(f"Unique Penn POD F001s (not shared):   {unique_count:,}")
print(f"Uniqueness rate:                      {(unique_count/penn_std_count*100):.1f}%")
print("="*60)

# Display sample of unique records (recompute id_type for display)
print("\nSample of unique Penn POD records:")
conservative_unique_penn_display = conservative_unique_penn.withColumn(
    "id_type",
    F.when(F.col("key").rlike("^[0-9X]{10,13}$"), F.lit("ISBN"))
     .when(F.col("key").rlike("^[0-9]{8,}$"), F.lit("OCLC"))
     .when(F.col("key").contains("_"), F.lit("MatchKey"))
     .when(F.col("key").rlike("^[A-Za-z0-9-]{5,}$"), F.lit("LCCN"))
     .otherwise(F.lit("OTHER"))
)
conservative_unique_penn_display.select("F001", "key", "id_type", "source").show(10, truncate=False)

print("\n✅ Conservative uniqueness analysis complete!")
print("💡 Next: Apply format filters (no 533 fields) for final refinement")

In [ ]:
# Additional Filtering - Reproduction Removal and HSP Exclusion
from pyspark.sql.functions import col, when, size, array_contains, collect_set, min
import pyspark.sql.functions as F

print("=== ADDITIONAL FILTERING FOR ACCURATE UNIQUENESS ===")
print("Removing reproductions and HSP records while preserving all unique physical items\n")

# Load the conservative unique records if not already loaded
if 'conservative_unique' not in locals():
    conservative_unique = spark.read.parquet(f"{output_dir}/conservative_unique_penn.parquet")

# Get the count of conservative unique records as our baseline
if 'conservative_unique_count' not in locals():
    conservative_unique_count = conservative_unique.select("F001").distinct().count()
    print(f"Conservative unique records loaded: {conservative_unique_count:,}\n")

# Load the full Penn records with all fields
penn_full = spark.read.parquet(f"{input_dir}/penn_penn-marc21.parquet")

# IMPORTANT: Ensure we only get unique F001s from penn_full to avoid duplicates
penn_full_unique = penn_full.dropDuplicates(["F001"])

# Join to get full records for conservative unique items
conservative_unique_full = conservative_unique.join(penn_full_unique, on="F001", how="inner")

# Verify the join didn't create duplicates
joined_count = conservative_unique_full.count()
if joined_count != conservative_unique_count:
    print(f"⚠️  WARNING: Join created duplicates! Expected {conservative_unique_count:,}, got {joined_count:,}")
    print("Deduplicating by F001...")
    conservative_unique_full = conservative_unique_full.dropDuplicates(["F001"])
    joined_count = conservative_unique_full.count()
    print(f"After deduplication: {joined_count:,} records\n")

print("=== FILTERING STEPS ===")
print("Note: NOT deduplicating by ISBN or OCLC - we want to count ALL unique physical items\n")

# Start with the full joined records
conservative_filtered = conservative_unique_full

print("📋 STEP 1: REMOVING REPRODUCTIONS (F533 FIELD)")
print("Filtering out records with reproduction notes...")

# Remove records with F533 (reproduction note)
if "F533" in conservative_filtered.columns:
    conservative_no_reproductions = conservative_filtered.filter(col("F533").isNull())
    no_repro_count = conservative_no_reproductions.count()
    removed_by_f533 = joined_count - no_repro_count
    
    print(f"  - Records before F533 filter: {joined_count:,}")
    print(f"  - Records after F533 filter: {no_repro_count:,}")
    print(f"  - Removed by F533 filter: {removed_by_f533:,}")
else:
    print("  - F533 field not found, skipping reproduction filter")
    conservative_no_reproductions = conservative_filtered
    no_repro_count = joined_count
    removed_by_f533 = 0

print("\n📋 STEP 2: REMOVING HSP (HISTORICAL SOCIETY OF PENNSYLVANIA) RECORDS")
print("Loading HSP exclusion list from file...")

# Load HSP F001 values from text file
hsp_file_path = "/home/jovyan/work/July-2025-PODParquet/hsp_removed_mmsid.txt"
try:
    # Read the HSP F001 values from the text file
    with open(hsp_file_path, 'r') as f:
        hsp_f001_list = [line.strip() for line in f if line.strip()]
    
    print(f"  - Loaded {len(hsp_f001_list):,} HSP F001 values from file")
    
    # Convert to DataFrame for efficient joining
    hsp_df = spark.createDataFrame([(f001,) for f001 in hsp_f001_list], ["F001"])
    
    # Remove HSP records using anti-join
    conservative_no_hsp = conservative_no_reproductions.join(
        hsp_df,
        on="F001",
        how="left_anti"
    )
    
    no_hsp_count = conservative_no_hsp.count()
    removed_by_hsp = no_repro_count - no_hsp_count
    
    print(f"  - Records before HSP filter: {no_repro_count:,}")
    print(f"  - Records after HSP filter: {no_hsp_count:,}")
    print(f"  - Removed by HSP filter: {removed_by_hsp:,}")
    
except FileNotFoundError:
    print(f"  ⚠️  WARNING: HSP file not found at {hsp_file_path}")
    print("  Falling back to pattern-based HSP detection...")
    
    # Fallback: Use pattern matching approach
    hsp_patterns = [
        "HSP",
        "Historical Society of Pennsylvania",
        "Hist Soc Penn",
        "Hist.Soc.Penn"
    ]
    
    # Create filter conditions for HSP detection
    hsp_filter_conditions = F.lit(False)
    
    # Check F710 (corporate name added entry)
    if "F710" in conservative_no_reproductions.columns:
        for pattern in hsp_patterns:
            hsp_filter_conditions = hsp_filter_conditions | \
                F.array_contains(F.transform(F.col("F710"), lambda x: F.upper(x)), pattern.upper())
    
    # Check F590 (local note)
    if "F590" in conservative_no_reproductions.columns:
        for pattern in hsp_patterns:
            hsp_filter_conditions = hsp_filter_conditions | \
                F.array_contains(F.transform(F.col("F590"), lambda x: F.upper(x)), pattern.upper())
    
    # Check F500 (general note)
    if "F500" in conservative_no_reproductions.columns:
        for pattern in hsp_patterns:
            hsp_filter_conditions = hsp_filter_conditions | \
                F.array_contains(F.transform(F.col("F500"), lambda x: F.upper(x)), pattern.upper())
    
    # Apply HSP filter
    conservative_no_hsp = conservative_no_reproductions.filter(~hsp_filter_conditions)
    no_hsp_count = conservative_no_hsp.count()
    removed_by_hsp = no_repro_count - no_hsp_count
    
    print(f"  - Records before HSP filter: {no_repro_count:,}")
    print(f"  - Records after HSP filter: {no_hsp_count:,}")
    print(f"  - Removed by HSP filter: {removed_by_hsp:,}")

# Calculate final statistics
print("\n=== FINAL FILTERED UNIQUENESS SUMMARY ===")
print(f"Conservative unique (starting point): {conservative_unique_count:,}")
print(f"After removing reproductions: {no_repro_count:,}")
print(f"After removing HSP records: {no_hsp_count:,}")

# Calculate filtering rate (what percentage of conservative unique records survived filtering)
filtering_rate = (no_hsp_count / conservative_unique_count * 100) if conservative_unique_count > 0 else 0
print(f"\nFiltering rate: {filtering_rate:.1f}% of conservative unique records retained")

print(f"\n📊 FILTERING IMPACT:")
print(f"  - Reproductions removed: {removed_by_f533:,}")
print(f"  - HSP records removed: {removed_by_hsp:,}")
print(f"  - Total filtered out: {conservative_unique_count - no_hsp_count:,}")
print(f"\n✅ PRESERVED: All unique physical items (no ISBN/OCLC deduplication)")

# Save the final filtered dataset
conservative_no_hsp.select("F001").write.mode("overwrite").parquet(
    f"{output_dir}/conservative_unique_penn_filtered.parquet"
)

print(f"\n✅ Additional filtering complete!")
print(f"Final filtered results saved to: {output_dir}/conservative_unique_penn_filtered.parquet")
print(f"\n💡 This dataset preserves ALL unique physical items Penn owns")
print(f"   Each physical copy/item is counted separately as intended")

# Update the unique_penn variable for downstream processing
unique_penn = conservative_no_hsp.select("F001")
unique_penn_count = no_hsp_count

# Store the baseline for use in next cell
total_penn = conservative_unique_count  # This is the correct baseline for the rate calculation

In [ ]:
# Clear Spark cache before Material Type Analysis
spark.catalog.clearCache()

# Material Type Analysis - CORRECTED VERSION
from pyspark.sql.functions import col, substring, when, concat, lit
import pyspark.sql.functions as F
import os

print("=== MATERIAL TYPE ANALYSIS (CORRECTED) ===")

# Load the CORRECT filtered unique Penn F001s (not the exploded dataset!)
print("\n📂 Loading the correct filtered unique Penn F001s...")

# Check which filtered dataset is available (most filtered to least)
filtered_paths = [
    f"{output_dir}/conservative_unique_penn_filtered_no_f035_hsp.parquet",
    f"{output_dir}/conservative_unique_penn_filtered.parquet",
    f"{output_dir}/conservative_unique_penn.parquet"
]

unique_penn_f001s = None
for path in filtered_paths:
    if os.path.exists(path):
        print(f"✅ Found filtered dataset: {os.path.basename(path)}")
        # Load just the F001s (this should be the filtered unique records)
        temp_df = spark.read.parquet(path)
        
        # If this has full MARC data, extract just F001s
        if "F001" in temp_df.columns:
            unique_penn_f001s = temp_df.select("F001").distinct()
        else:
            # This might already be just F001s
            unique_penn_f001s = temp_df
        break

if unique_penn_f001s is None:
    raise FileNotFoundError("No filtered unique Penn dataset found! Please run the filtering cells first.")

# Count the ACTUAL unique Penn records
unique_penn_count = unique_penn_f001s.count()
print(f"📊 Filtered unique Penn F001s: {unique_penn_count:,}")

# Verify this is reasonable (should be < 5,362,031 after HSP filtering)
if unique_penn_count > 5_362_031:
    print("❌ ERROR: Count is too high! This appears to be the exploded dataset.")
    print("   Expected: < 5,362,031 (after HSP filtering)")
    print("   Got: {unique_penn_count:,}")
    raise ValueError("Using wrong dataset - this is the exploded dataset, not unique F001s")

# Load the FULL Penn MARC records (the original dataset with all fields)
penn_full_path = f"{input_dir}/penn_penn-marc21.parquet"
penn_full = spark.read.parquet(penn_full_path)
print(f"\n✅ Loaded full Penn MARC records: {penn_full.count():,} total records")

# Check for Leader field
if "FLDR" in penn_full.columns:
    LEADER_FIELD = "FLDR"
elif "LDR" in penn_full.columns:
    LEADER_FIELD = "LDR"
else:
    raise ValueError("No Leader field found!")

print(f"✅ Using Leader field: {LEADER_FIELD}")

# JOIN to get only the filtered unique records with full MARC data
print(f"\n🔄 Joining {unique_penn_count:,} unique F001s with full MARC records...")
unique_penn_full = penn_full.join(unique_penn_f001s, on="F001", how="inner")

# Verify the join
joined_count = unique_penn_full.count()
print(f"✅ Joined records: {joined_count:,}")

if joined_count != unique_penn_count:
    print(f"⚠️ WARNING: Expected {unique_penn_count:,} but got {joined_count:,}")
    if joined_count > unique_penn_count:
        print("   There might be duplicate F001s in penn_full. Deduplicating...")
        unique_penn_full = unique_penn_full.dropDuplicates(["F001"])
        joined_count = unique_penn_full.count()
        print(f"   After dedup: {joined_count:,} records")

# Apply material type categorization
unique_penn_with_material_type = (unique_penn_full
    .withColumn("record_type", substring(col(LEADER_FIELD), 7, 1))
    .withColumn("bib_level", substring(col(LEADER_FIELD), 8, 1))
    .withColumn("material_category", 
        when((col("record_type") == "a") & (col("bib_level").isin("m")), "print_book")
        .when((col("record_type") == "a") & (col("bib_level").isin("s")), "print_serial")
        .when((col("record_type") == "c"), "print_music")
        .when((col("record_type") == "e"), "print_maps")
        .when(col("record_type") == "m", "electronic_resource")
        .when(col("record_type").isin("g", "k"), "visual_material")
        .when(col("record_type") == "i", "audio_material")
        .otherwise("other")
    )
    .withColumn("is_print", 
        col("material_category").isin("print_book", "print_serial", "print_music", "print_maps")
    )
)

# Cache for analysis
unique_penn_with_material_type.cache()

# Get statistics
print("\n📊 Material Type Distribution for FILTERED UNIQUE Penn records:")
material_stats = unique_penn_with_material_type.groupBy("material_category", "is_print").count().collect()

material_counts_dict = {}
print_count = 0
non_print_count = 0

for row in material_stats:
    material_counts_dict[row["material_category"]] = row["count"]
    if row["is_print"]:
        print_count += row["count"]
    else:
        non_print_count += row["count"]

total_unique = print_count + non_print_count

# Display distribution
for category, count in sorted(material_counts_dict.items(), key=lambda x: x[1], reverse=True):
    print(f"  {category}: {count:,}")

print(f"\n📊 Summary:")
print(f"  Total FILTERED UNIQUE Penn records analyzed: {total_unique:,}")
print(f"  Print materials: {print_count:,} ({print_count/total_unique*100:.1f}%)")
print(f"  Non-print materials: {non_print_count:,} ({non_print_count/total_unique*100:.1f}%)")

# Save the correctly filtered datasets
print("\n💾 Saving corrected datasets...")
unique_penn_with_material_type.write.mode("overwrite").parquet(f"{output_dir}/unique_penn_with_material_type.parquet")

print_only_df = unique_penn_with_material_type.filter(col("is_print") == True)
print_only_df.write.mode("overwrite").parquet(f"{output_dir}/physical_books_no_533.parquet")

print(f"\n✅ Material type analysis complete!")
print(f"   Analyzed {total_unique:,} UNIQUE Penn records")
print(f"   Starting from {unique_penn_count:,} filtered F001s")
print(f"   (NOT the 30M exploded dataset!)")

# Unpersist
unique_penn_with_material_type.unpersist()

In [ ]:
# Clear cache and adjust broadcast settings to prevent timeout
spark.catalog.clearCache()

# Disable broadcast joins temporarily to prevent timeout
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

# Increase broadcast timeout
spark.conf.set("spark.sql.broadcastTimeout", "1200")  # 20 minutes

from pyspark.sql.functions import col, regexp_replace, lower, trim, concat_ws, when, regexp_extract, size, array_join, slice, split
import pyspark.sql.functions as F
import json
from datetime import datetime
import builtins

# Define the create_match_key_spark function if not already defined
if 'create_match_key_spark' not in globals():
    def create_match_key_spark(df):
        """
        Create match keys for the dataframe
        """
        # Check if required columns exist
        required_cols = ['F245', 'F250', 'F260', 'F264']
        existing_cols = [col for col in required_cols if col in df.columns]
        
        # Extract publication year from F260 or F264
        df = df.withColumn("pub_year",
            F.coalesce(
                F.when(F.col("F260").isNotNull() & (F.size(F.col("F260")) > 0),
                    F.regexp_extract(F.col("F260").getItem(0), "(1[5-9][0-9]{2}|20[0-9]{2})", 1)
                ) if "F260" in df.columns else F.lit(None),
                F.when(F.col("F264").isNotNull() & (F.size(F.col("F264")) > 0),
                    F.regexp_extract(F.col("F264").getItem(0), "(1[5-9][0-9]{2}|20[0-9]{2})", 1)
                ) if "F264" in df.columns else F.lit(None)
            )
        )
        
        # Normalize title
        df = df.withColumn("title_normalized",
            F.when(F.col("F245").isNotNull(),
                F.regexp_replace(
                    F.regexp_replace(
                        F.lower(F.trim(F.col("F245"))),
                        "^(the|a|an|le|la|los|las|el|die|der|das|den|det)\\s+", ""
                    ),
                    "[^a-z0-9\\s]", ""
                )
            ).otherwise("")
        )
        
        # Create match key
        df = df.withColumn("match_key", 
            F.concat_ws("_",
                F.col("title_normalized"),
                F.when(F.col("F250").isNotNull() & (F.size(F.col("F250")) > 0),
                    F.regexp_replace(
                        F.lower(F.col("F250").getItem(0)), 
                        "(\\d+)(?:st|nd|rd|th)?\\s*(?:ed|edition)", "$1 ed"
                    )
                ).otherwise("") if "F250" in df.columns else F.lit(""),
                F.coalesce(F.col("pub_year"), F.lit(""))
            )
        )
        
        return df

# Define output directory if not already defined
if 'output_dir' not in locals():
    output_dir = "/home/jovyan/work/July-2025-PODParquet/pod-processing-outputs"

# Load print materials dataset if not already loaded
if 'print_only_df' not in locals() or print_only_df is None:
    print("Loading print materials dataset...")
    print_only_df_raw = spark.read.parquet(f"{output_dir}/physical_books_no_533.parquet")
    metadata_cols = ["processing_date", "source_file", "data_currency_warning"]
    existing_metadata_cols = [col for col in metadata_cols if col in print_only_df_raw.columns]
    if existing_metadata_cols:
        print(f"Dropping metadata columns: {existing_metadata_cols}")
        print_only_df = print_only_df_raw.drop(*existing_metadata_cols)
    else:
        print_only_df = print_only_df_raw
else:
    print("Using existing print_only_df DataFrame")

# Add match keys to the full print dataset using the existing function
print_only_df_with_keys = create_match_key_spark(print_only_df)

# Verify match keys were created
match_key_stats = print_only_df_with_keys.select(
    F.count("*").alias("total_records"),
    F.sum(F.when(F.col("match_key").isNotNull() & (F.col("match_key") != ""), 1).otherwise(0)).alias("records_with_match_key"),
    F.avg(F.length("match_key")).alias("avg_match_key_length")
).collect()[0]

print(f"\nMatch key generation results:")
print(f"  - Total records: {match_key_stats['total_records']:,}")
print(f"  - Records with match key: {match_key_stats['records_with_match_key']:,} ({match_key_stats['records_with_match_key']/match_key_stats['total_records']*100:.1f}%)")
print(f"  - Average match key length: {match_key_stats['avg_match_key_length']:.1f} characters")

# Cache the full dataset for performance
print_only_df_with_keys.cache()

# Save the full print dataset with match keys
print_only_df_with_keys.write.mode("overwrite").parquet(f"{output_dir}/physical_books_no_533.parquet")

print("\n✅ Match keys added to full print dataset (no sampling)")
print(f"Results saved to {output_dir}/physical_books_no_533.parquet")

In [ ]:
# Enhanced RDA filtering with OCLC numbers included for HathiTrust checking

from pyspark.sql import functions as F
import pandas as pd
import builtins

# Define the extract_oclc_number_enhanced function if not already defined
if 'extract_oclc_number_enhanced' not in globals():
    def extract_oclc_number_enhanced(df):
        """
        ENHANCED: Extract OCLC numbers from F035 field with ALL common patterns
        Handles ocm, ocn, on prefixes and leading zeros
        """
        return df.withColumn("oclc_number",
            F.when(F.col("F035").isNotNull() & (F.size(F.col("F035")) > 0),
                F.regexp_extract(
                    F.concat_ws(" ", F.col("F035")),
                    "\\(OCoLC\\)(?:ocm|ocn|on)?0*([0-9]+)",  # Handles prefixes AND leading zeros
                    1
                )
            )
        )

print("=== EXPORTING PHYSICAL BOOKS TO EXCEL WITH ENHANCED RDA FILTERING ===")
print("Reading from existing parquet file and applying comprehensive RDA filtering...\n")

# Read the existing physical_books_no_533.parquet
physical_books_df = spark.read.parquet(f"{output_dir}/physical_books_no_533.parquet")

# Check if OCLC numbers already exist (from previous processing)
if "oclc_number" in physical_books_df.columns:
    print("✅ OCLC numbers already present in dataset")
else:
    print("📋 Extracting OCLC numbers...")
    physical_books_df = extract_oclc_number_enhanced(physical_books_df)

# Get initial count
initial_count = physical_books_df.count()
print(f"Initial records: {initial_count:,}")

# Apply comprehensive RDA filtering
print("\n📋 APPLYING COMPREHENSIVE RDA FILTERING")
print("Checking multiple RDA fields with false positive prevention...")

# Create electronic filter function
def create_electronic_filter(df):
    """
    Create a comprehensive filter for electronic resources while avoiding false positives
    """
    # Primary electronic indicators (high confidence)
    primary_electronic = (
        # 337 $a = computer (media type)
        (F.col("F337").isNotNull() & 
         F.array_contains(
             F.transform(F.col("F337"), lambda x: F.lower(x)), 
             F.lit("computer")
         )) |
        # 338 $a = online resource or other computer carriers
        (F.col("F338").isNotNull() & 
         (F.array_contains(F.transform(F.col("F338"), lambda x: F.lower(x)), F.lit("online")) |
          F.array_contains(F.transform(F.col("F338"), lambda x: F.lower(x)), F.lit("computer disc")) |
          F.array_contains(F.transform(F.col("F338"), lambda x: F.lower(x)), F.lit("computer chip")) |
          F.array_contains(F.transform(F.col("F338"), lambda x: F.lower(x)), F.lit("computer tape"))
         ))
    )
    
    # Check for electronic format in 300 field (physical description)
    electronic_300 = F.lit(False)
    if "F300" in df.columns:
        electronic_300 = (
            F.col("F300").isNotNull() & 
            F.array_contains(
                F.transform(F.col("F300"), lambda x: F.lower(x)), 
                F.lit("online resource")
            )
        )
    
    # Combine filters with OR logic
    return primary_electronic | electronic_300

# Apply the comprehensive filter
electronic_filter = create_electronic_filter(physical_books_df)
physical_books_filtered = physical_books_df.filter(~electronic_filter)

# Count after filtering
filtered_count = physical_books_filtered.count()
removed_by_rda = initial_count - filtered_count

print(f"\nEnhanced RDA filtering results:")
print(f"  - Records before RDA filter: {initial_count:,}")
print(f"  - Records after RDA filter: {filtered_count:,}")
print(f"  - Removed by RDA filter: {removed_by_rda:,}")

# EXTRACT OCLC NUMBERS for the filtered dataset (if not already present)
print("\n📋 Checking OCLC numbers for HathiTrust checking...")
physical_books_with_oclc = physical_books_filtered

# Check if oclc_number already exists, if not extract it
if "oclc_number" not in physical_books_with_oclc.columns:
    print("  - Extracting OCLC numbers from F035 field...")
    physical_books_with_oclc = extract_oclc_number_enhanced(physical_books_with_oclc)

# Count records with OCLC numbers
oclc_count = physical_books_with_oclc.filter(
    F.col("oclc_number").isNotNull() & (F.col("oclc_number") != "")
).count()
print(f"  - Records with OCLC numbers: {oclc_count:,} ({oclc_count/filtered_count*100:.1f}%)")

# Prepare data for Excel export INCLUDING OCLC NUMBERS
print("\n📋 Preparing data for Excel export with OCLC numbers...")
physical_books_excel = physical_books_with_oclc.select(
    "F001",
    # OCLC number - ADDED FOR HATHITRUST
    F.when(F.col("oclc_number").isNotNull(), F.col("oclc_number")).otherwise("").alias("OCLC"),
    # F020 is an array - get first ISBN if available
    F.when(F.col("F020").isNotNull() & (F.size(F.col("F020")) > 0), 
           F.col("F020").getItem(0)).otherwise("").alias("ISBN"),
    F.col("F010").alias("LCCN"),
    F.col("F245").alias("Title"),
    # F250 is an array - get first edition statement if available
    F.when(F.col("F250").isNotNull() & (F.size(F.col("F250")) > 0), 
           F.col("F250").getItem(0)).otherwise("").alias("Edition"),
    # F260 is an array - get first publication info if available
    F.when(F.col("F260").isNotNull() & (F.size(F.col("F260")) > 0), 
           F.col("F260").getItem(0)).otherwise("").alias("Publication"),
    # Also check F264 for publication info if F260 is empty
    F.when(
        (F.col("F260").isNull() | (F.size(F.col("F260")) == 0)) & 
        F.col("F264").isNotNull() & (F.size(F.col("F264")) > 0),
        F.col("F264").getItem(0)
    ).otherwise("").alias("Publication_264"),
    "material_category",
    "match_key"
)

# Check the size before converting to pandas
print(f"\n⚠️  Dataset has {filtered_count:,} records - checking Excel limits...")

# Excel has a limit of 1,048,576 rows
EXCEL_ROW_LIMIT = 1048576

if filtered_count > EXCEL_ROW_LIMIT:
    print(f"❌ Dataset too large for single Excel file ({filtered_count:,} > {EXCEL_ROW_LIMIT:,})")
    print("\n📋 OPTIONS:")
    
    # Option 1: Sample the data
    print("\n1️⃣ OPTION 1: Create a sample Excel file (first 1M records)")
    sample_df = physical_books_excel.limit(EXCEL_ROW_LIMIT - 1)  # -1 for header
    pandas_df_sample = sample_df.toPandas()
    
    # Combine Publication columns
    if 'Publication_264' in pandas_df_sample.columns:
        pandas_df_sample['Publication'] = pandas_df_sample.apply(
            lambda row: row['Publication'] if row['Publication'] else row['Publication_264'], 
            axis=1
        )
        pandas_df_sample = pandas_df_sample.drop('Publication_264', axis=1)
    
    # Save sample to Excel
    excel_sample_path = f"{output_dir}/physical_books_no_533_no_electronic_with_oclc_SAMPLE.xlsx"
    print(f"   Writing sample to: {excel_sample_path}")
    
    with pd.ExcelWriter(excel_sample_path, engine='openpyxl') as writer:
        pandas_df_sample.to_excel(writer, sheet_name='Physical Books Sample', index=False)
        
        # Auto-adjust column widths
        worksheet = writer.sheets['Physical Books Sample']
        for column in pandas_df_sample:
            column_length = builtins.max(pandas_df_sample[column].astype(str).map(len).max(), len(column))
            col_idx = pandas_df_sample.columns.get_loc(column)
            if col_idx < 26:  # Only handle first 26 columns (A-Z)
                worksheet.column_dimensions[chr(65 + col_idx)].width = builtins.min(column_length + 2, 50)
    
    print(f"   ✅ Sample Excel created with {len(pandas_df_sample):,} records")
    
    # Option 2: Save as CSV (no row limit)
    print("\n2️⃣ OPTION 2: Save full dataset as CSV (no row limit)")
    csv_path = f"{output_dir}/physical_books_no_533_no_electronic_with_oclc.csv"
    
    # Convert to pandas in chunks to avoid memory issues
    print(f"   Converting to CSV: {csv_path}")
    pandas_df_full = physical_books_excel.toPandas()
    
    # Combine Publication columns
    if 'Publication_264' in pandas_df_full.columns:
        pandas_df_full['Publication'] = pandas_df_full.apply(
            lambda row: row['Publication'] if row['Publication'] else row['Publication_264'], 
            axis=1
        )
        pandas_df_full = pandas_df_full.drop('Publication_264', axis=1)
    
    pandas_df_full.to_csv(csv_path, index=False)
    print(f"   ✅ CSV created with all {len(pandas_df_full):,} records")
    
    # Always save the parquet version
    print("\n3️⃣ OPTION 3: Full dataset saved as Parquet (recommended for large data)")
    parquet_path = f"{output_dir}/physical_books_no_533_no_electronic_with_oclc.parquet"
    physical_books_with_oclc.write.mode("overwrite").parquet(parquet_path)
    print(f"   ✅ Parquet saved: {parquet_path}")
    
    print(f"\n📊 SUMMARY:")
    print(f"   - Total records: {filtered_count:,}")
    print(f"   - Records with OCLC: {oclc_count:,}")
    print(f"   - Electronic resources removed: {removed_by_rda:,}")
    print(f"   - Sample Excel: {excel_sample_path} (first 1M records)")
    print(f"   - Full CSV: {csv_path} (all records)")
    print(f"   - Full Parquet: {parquet_path} (all records)")
    
else:
    # Dataset fits in Excel - proceed normally
    print(f"✅ Dataset fits in Excel ({filtered_count:,} < {EXCEL_ROW_LIMIT:,})")
    
    # Convert to Pandas DataFrame
    print("\nConverting to Pandas DataFrame...")
    pandas_df = physical_books_excel.toPandas()
    
    # Combine Publication and Publication_264 if needed
    if 'Publication_264' in pandas_df.columns:
        pandas_df['Publication'] = pandas_df.apply(
            lambda row: row['Publication'] if row['Publication'] else row['Publication_264'], 
            axis=1
        )
        pandas_df = pandas_df.drop('Publication_264', axis=1)
    
    # Save to Excel
    excel_path = f"{output_dir}/physical_books_no_533_no_electronic_with_oclc.xlsx"
    print(f"\nWriting to Excel file: {excel_path}")
    
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        pandas_df.to_excel(writer, sheet_name='Physical Books', index=False)
        
        # Auto-adjust column widths
        worksheet = writer.sheets['Physical Books']
        for column in pandas_df:
            column_length = builtins.max(pandas_df[column].astype(str).map(len).max(), len(column))
            col_idx = pandas_df.columns.get_loc(column)
            if col_idx < 26:  # Only handle first 26 columns (A-Z)
                worksheet.column_dimensions[chr(65 + col_idx)].width = builtins.min(column_length + 2, 50)
    
    print(f"\n✅ Enhanced Excel export complete with OCLC numbers!")
    print(f"   - Output file: {excel_path}")
    print(f"   - Records exported: {filtered_count:,}")
    print(f"   - Records with OCLC: {oclc_count:,}")
    print(f"   - Electronic resources removed: {removed_by_rda:,}")
    
    # Also save a parquet version
    if removed_by_rda > 0 or "oclc_number" not in physical_books_filtered.columns:
        print(f"\n📋 Saving enhanced RDA-filtered data with OCLC to parquet...")
        physical_books_with_oclc.write.mode("overwrite").parquet(
            f"{output_dir}/physical_books_no_533_no_electronic_with_oclc.parquet"
        )
        print(f"   - Updated parquet: {output_dir}/physical_books_no_533_no_electronic_with_oclc.parquet")

# Show sample of records with OCLC numbers (works for both cases)
print("\n📋 Sample records with OCLC numbers:")
sample_preview = physical_books_excel.filter(F.col("OCLC") != "").limit(5).toPandas()
if len(sample_preview) > 0:
    print(sample_preview[['F001', 'OCLC', 'ISBN', 'Title']].to_string(index=False))
else:
    print("No records with OCLC numbers found in sample")

print("\n📋 FINAL OUTPUT INCLUDES:")
print("✅ F001 (MMS ID)")
print("✅ OCLC (for HathiTrust checking)")
print("✅ ISBN")
print("✅ LCCN")
print("✅ Title")
print("✅ Edition")
print("✅ Publication")
print("✅ Material Category")
print("✅ Match Key")

In [ ]:
# Safety: clear Spark cache if Spark is available (prevents stale cached DataFrames)
try:
    _ = spark
    spark.catalog.clearCache()
    print("🧹 Spark cache cleared")
except NameError:
    # Spark not in this kernel/session
    pass

# Skipping the Harvard API check in favor of BD and Alma APIs

In [17]:
# Harvard API prep: build worklist using prior Harvard results as exclusion (no API calls yet)
print("\n" + "="*60)
print("HARVARD API PREP — EXCLUDE PRIOR RESULTS (NO NETWORK CALLS)")
print("="*60)

import os
import pandas as pd
import json
import glob
import csv as _csv
import pickle
from datetime import datetime

# Control flags
FORCE_FULL_RUN = False                 # If True, ignore exclusions and process all current records
USE_BASELINE_FOR_DELTA = False         # If True, also subtract a July baseline; default False per user request

# Paths
EXACT_BASE_DIR = '/home/jovyan/work/July-2025-PODParquet/pod-processing-outputs'
EXACT_CURRENT_PATH = os.path.join(EXACT_BASE_DIR, 'physical_books_no_533_no_electronic_with_oclc.csv')
EXACT_BASELINE_PATH = os.path.join(EXACT_BASE_DIR, 'physical_books_no_533_with_match_keys.csv')

# Helper: ensure F001 exists and standard columns are present

def _normalize_id_column(df: pd.DataFrame) -> pd.DataFrame:
    cols = {c.lower().strip(): c for c in df.columns}
    if 'f001' in cols:
        df['F001'] = df[cols['f001']].astype(str).str.strip()
        return df
    for candidate in ['mmsid', 'mms_id', 'mms id', 'id']:
        if candidate in cols:
            df['F001'] = df[cols[candidate]].astype(str).str.strip()
            return df
    if 'F001' not in df.columns:
        df['F001'] = ''
    return df


def _coerce_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = _normalize_id_column(df)
    for col in ['OCLC', 'ISBN', 'Title']:
        if col not in df.columns:
            df[col] = ''
    for col in ['F001', 'OCLC', 'ISBN']:
        df[col] = df[col].astype(str).str.strip()
    return df


def _read_any_table(path: str, usecols=None) -> pd.DataFrame:
    ext = os.path.splitext(path)[1].lower()
    if ext in ['.csv', '.txt', '.tsv']:
        sep = ',' if ext == '.csv' else '\t'
        if usecols and isinstance(usecols, (list, set)):
            wanted = {c.strip().lower() for c in usecols}
            usecols_callable = lambda c: c is not None and c.strip().lower() in wanted
        else:
            usecols_callable = usecols
        try:
            return pd.read_csv(
                path, sep=sep, usecols=usecols_callable, dtype=str, low_memory=False,
                quotechar='"', escapechar='\\', engine='c', on_bad_lines='skip'
            )
        except Exception:
            return pd.read_csv(
                path, sep=sep, dtype=str, low_memory=False,
                quotechar='"', escapechar='\\', engine='c', on_bad_lines='skip'
            )
    if ext in ['.parquet']:
        return pd.read_parquet(path, columns=usecols if usecols else None)
    if ext in ['.xlsx', '.xls']:
        return pd.read_excel(path, usecols=usecols, dtype=str)
    raise ValueError(f"Unsupported file type: {path}")


def _count_physical_lines(path: str, max_lines: int | None = None) -> int:
    cnt = 0
    with open(path, 'rb') as f:
        for i, _ in enumerate(f, 1):
            if max_lines and i >= max_lines:
                return i
        cnt = i if 'i' in locals() else 0
    return cnt


def load_current_dataset(base_dir: str) -> pd.DataFrame:
    p = EXACT_CURRENT_PATH
    if not os.path.exists(p):
        raise FileNotFoundError(p)
    print(f"📄 Current dataset: {p}")
    print(f"   (physical lines ≈ {_count_physical_lines(p):,})")
    df = _read_any_table(p, usecols=['F001', 'OCLC', 'ISBN', 'Title'])
    df = _coerce_columns(df)
    df = df[['F001', 'OCLC', 'ISBN', 'Title']].dropna(subset=['F001'])
    print(f"   → Loaded {len(df):,} rows (logical records)")
    print(f"   → Unique MMS IDs: {df['F001'].nunique():,}")
    return df


def load_baseline_dataset(base_dir: str) -> pd.DataFrame:
    p = EXACT_BASELINE_PATH
    if not os.path.exists(p):
        return pd.DataFrame({'F001': []})
    print(f"📄 Baseline dataset: {p}")
    print(f"   (physical lines ≈ {_count_physical_lines(p):,})")
    df = _read_any_table(p, usecols={'F001', 'mmsid', 'mms_id', 'mms id', 'id'})
    df = _coerce_columns(df)
    df = df[['F001']].dropna(subset=['F001']).drop_duplicates()
    print(f"   → Unique MMS IDs (baseline): {df['F001'].nunique():,}")
    return df


def _add_ids_from_json_file(path: str, already: set):
    try:
        with open(path, 'r') as f:
            data = json.load(f)
        # Flexible extraction
        def collect(obj):
            if isinstance(obj, dict):
                # Common fields: 'F001', 'mms_id', 'id'
                for k in ['F001','mms_id','mmsid','id']:
                    if k in obj and obj[k]:
                        already.add(str(obj[k]).strip())
                for v in obj.values():
                    collect(v)
            elif isinstance(obj, list):
                for item in obj:
                    collect(item)
        collect(data)
        print(f"   → Added IDs from {os.path.basename(path)} (total now {len(already):,})")
    except Exception as e:
        print(f"   ! Failed to parse {os.path.basename(path)}: {e}")


def _add_ids_from_pickle(path: str, already: set):
    try:
        with open(path, 'rb') as f:
            obj = pickle.load(f)
        # Try common shapes: list[dict], dict[str, Any], set/list[str]
        if isinstance(obj, (set, list)):
            for item in obj:
                if isinstance(item, dict):
                    val = item.get('F001') or item.get('mms_id') or item.get('id')
                    if val:
                        already.add(str(val).strip())
                elif isinstance(item, (str, int)):
                    already.add(str(item).strip())
        elif isinstance(obj, dict):
            for v in obj.values():
                if isinstance(v, (list, set)):
                    for item in v:
                        if isinstance(item, dict):
                            val = item.get('F001') or item.get('mms_id') or item.get('id')
                            if val:
                                already.add(str(val).strip())
                        elif isinstance(item, (str, int)):
                            already.add(str(item).strip())
        print(f"   → Added IDs from {os.path.basename(path)} (total now {len(already):,})")
    except Exception as e:
        print(f"   ! Failed to parse pickle {os.path.basename(path)}: {e}")


def load_already_checked_ids(base_dir: str) -> set:
    already = set()
    # Known Harvard outputs to use as exclusion sources
    files = [
        os.path.join(base_dir, 'harvard_api_full_checkpoint.json'),
        os.path.join(base_dir, 'harvard_api_full_results.csv'),
        os.path.join(base_dir, 'harvard_api_full_results.parquet'),
        os.path.join(base_dir, 'harvard_api_results_complete_*.csv'),
        os.path.join(base_dir, 'harvard_check_results_verified_fixed.json'),
        os.path.join(base_dir, 'harvard_check_results_verified.json'),
        os.path.join(base_dir, 'harvard_check_results.json'),
        os.path.join(base_dir, 'harvard_check_checkpoint_v3.pkl'),
    ]

    for p in files:
        if '*' in p:
            for csv_path in glob.glob(p):
                try:
                    df = pd.read_csv(csv_path, dtype=str, low_memory=False)
                    df = _normalize_id_column(df)
                    ids = set(df['F001'].astype(str).str.strip())
                    already |= ids
                    print(f"   → Prior results exclude: {len(ids):,} from {os.path.basename(csv_path)}")
                except Exception as e:
                    print(f"   ! Failed to read {os.path.basename(csv_path)}: {e}")
            continue
        if not os.path.exists(p):
            continue
        ext = os.path.splitext(p)[1].lower()
        if ext == '.json':
            _add_ids_from_json_file(p, already)
        elif ext == '.csv':
            try:
                df = pd.read_csv(p, dtype=str, low_memory=False)
                df = _normalize_id_column(df)
                ids = set(df['F001'].astype(str).str.strip())
                already |= ids
                print(f"   → Prior results exclude: {len(ids):,} from {os.path.basename(p)}")
            except Exception as e:
                print(f"   ! Failed to read {os.path.basename(p)}: {e}")
        elif ext == '.parquet':
            try:
                df = pd.read_parquet(p, columns=None)
                df = _normalize_id_column(df)
                ids = set(df['F001'].astype(str).str.strip())
                already |= ids
                print(f"   → Prior results exclude: {len(ids):,} from {os.path.basename(p)}")
            except Exception as e:
                print(f"   ! Failed to read {os.path.basename(p)}: {e}")
        elif ext == '.pkl':
            _add_ids_from_pickle(p, already)
    return already

# 1) Load current inputs
current_df = load_current_dataset(EXACT_BASE_DIR)
current_ids = set(current_df['F001'].astype(str))

# 2) Build exclusion set from prior Harvard results
print("\nGathering already-processed MMS IDs from prior Harvard outputs…")
already_checked = load_already_checked_ids(EXACT_BASE_DIR)
print(f"   Total already-checked IDs: {len(already_checked):,}")

# 3) Optional: load baseline if requested (off by default)
baseline_ids = set()
if USE_BASELINE_FOR_DELTA and not FORCE_FULL_RUN:
    baseline_df = load_baseline_dataset(EXACT_BASE_DIR)
    baseline_ids = set(baseline_df['F001'].astype(str)) if len(baseline_df) else set()

# 4) Compute worklist according to flags
if FORCE_FULL_RUN:
    work_ids = current_ids
    mode = 'full-run (ignore prior results/baseline)'
elif USE_BASELINE_FOR_DELTA and baseline_ids:
    # subtract both July baseline and prior results
    work_ids = (current_ids - baseline_ids) - already_checked
    mode = 'delta vs July baseline minus prior results'
else:
    # default: October-only style — subtract prior results only
    work_ids = current_ids - already_checked
    mode = 'current minus prior Harvard results (October-only)'

print("\n📊 Worklist composition:")
print(f"   Current unique MMS IDs:  {len(current_ids):,}")
if USE_BASELINE_FOR_DELTA and baseline_ids:
    print(f"   Baseline unique MMS IDs: {len(baseline_ids):,}")
print(f"   Already-checked IDs:     {len(already_checked):,}")
print(f"   Mode:                    {mode}")
print(f"   Final to-check:          {len(work_ids):,} IDs")

# 5) Persist worklist
worklist_df = current_df[current_df['F001'].isin(work_ids)].copy()
worklist_df = worklist_df[['F001', 'OCLC', 'ISBN', 'Title']].drop_duplicates('F001')

print("\n✅ HARVARD WORKLIST READY")
print(f"   Final to-check count: {len(worklist_df):,} MMS IDs")

worklist_path = os.path.join(EXACT_BASE_DIR, 'harvard_api_worklist.csv')
worklist_df.to_csv(worklist_path, index=False)
print(f"   Saved worklist CSV: {worklist_path}")

sample_path = os.path.join(EXACT_BASE_DIR, 'generated_api_sample.csv')
worklist_df.sample(min(1000, len(worklist_df)), random_state=42).to_csv(sample_path, index=False)
print(f"   Saved sample:       {sample_path}")

manifest = {
    'generated_at': datetime.now().isoformat(),
    'mode': mode,
    'current_count_unique': len(current_ids),
    'baseline_count_unique': len(baseline_ids) if (USE_BASELINE_FOR_DELTA and baseline_ids) else None,
    'already_checked_excluded': len(already_checked),
    'final_to_check': len(worklist_df),
    'inputs': {
        'current_path': EXACT_CURRENT_PATH,
        'baseline_path': EXACT_BASELINE_PATH if USE_BASELINE_FOR_DELTA else None,
    },
    'artifacts': {
        'worklist_csv': 'harvard_api_worklist.csv',
        'sample_csv': 'generated_api_sample.csv'
    }
}
with open(os.path.join(EXACT_BASE_DIR, 'harvard_api_prep_manifest.json'), 'w') as f:
    json.dump(manifest, f, indent=2)
print(f"   Saved manifest:     {os.path.join(EXACT_BASE_DIR, 'harvard_api_prep_manifest.json')}")

print("\n💡 Next: run the Harvard API against harvard_api_worklist.csv with checkpointing.")

# Final Check that these Penn MMSIDs exist and are not held in HSP

In [ ]:
# Alma API check for Penn-unique items (using reconstructed worklist)
print("\n" + "="*60)
print("ALMA API CHECK FOR PENN-UNIQUE ITEMS")
print("="*60)

import pandas as pd
import requests
import time
import json
from datetime import datetime
import xml.etree.ElementTree as ET
from urllib.parse import quote
import re
import os

# Configuration flags
RUN_MODE = "full"  # "demo" or "full"
CHECKPOINT_INTERVAL = 100  # Save progress every N records
RATE_LIMIT_DELAY = 0.05  # Delay between API calls in seconds

# Use the RECONSTRUCTED worklist as input (the correct filtered dataset)
worklist_path = "/home/jovyan/work/July-2025-PODParquet/pod-processing-outputs/harvard_api_worklist_reconstructed.csv"

class AlmaAPIChecker:
    """Check if MMS IDs exist in Penn's Alma catalog using SRU API"""
    
    def __init__(self):
        self.base_url = "https://upenn.alma.exlibrisgroup.com/view/sru/01UPENN_INST"
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Penn-Library-Research/1.0',
            'Accept': 'application/xml'
        })
        self.last_request_time = 0
        self.rate_limit_delay = RATE_LIMIT_DELAY
        
    def _rate_limit(self):
        """Enforce rate limiting between API calls"""
        current_time = time.time()
        time_since_last = current_time - self.last_request_time
        if time_since_last < self.rate_limit_delay:
            time.sleep(self.rate_limit_delay - time_since_last)
        self.last_request_time = time.time()
    
    def check_mms_exists(self, mms_id):
        """
        Check if an MMS ID exists in Alma using SRU
        Returns: (exists, record_count, error_message)
        """
        self._rate_limit()
        
        try:
            # Clean the MMS ID - ensure it's a string
            mms_id_clean = str(mms_id).strip()
            
            # Construct SRU query for MMS ID
            query = f'alma.mms_id={mms_id_clean}'
            
            params = {
                'version': '1.2',
                'operation': 'searchRetrieve',
                'query': query,
                'maximumRecords': '1',
                'recordSchema': 'marcxml'
            }
            
            response = self.session.get(self.base_url, params=params, timeout=30)
            
            if response.status_code == 200:
                # Parse XML response
                root = ET.fromstring(response.content)
                
                # Define namespace
                ns = {
                    'srw': 'http://www.loc.gov/zing/srw/',
                    'marc': 'http://www.loc.gov/MARC21/slim'
                }
                
                # Get number of records
                num_records_elem = root.find('.//srw:numberOfRecords', ns)
                if num_records_elem is not None:
                    num_records = int(num_records_elem.text)
                    exists = num_records > 0
                    return exists, num_records, None
                else:
                    return False, 0, "Could not parse response"
                    
            else:
                return False, 0, f"HTTP {response.status_code}"
                
        except requests.exceptions.Timeout:
            return False, 0, "Timeout"
        except ET.ParseError as e:
            return False, 0, f"XML parse error: {str(e)}"
        except Exception as e:
            return False, 0, str(e)
    
    def check_batch(self, mms_ids, checkpoint_file=None):
        """
        Check a batch of MMS IDs with checkpoint support
        """
        results = []
        
        # Load checkpoint if exists
        start_idx = 0
        if checkpoint_file and os.path.exists(checkpoint_file):
            try:
                with open(checkpoint_file, 'r') as f:
                    checkpoint = json.load(f)
                    start_idx = checkpoint.get('last_index', 0)
                    results = checkpoint.get('results', [])
                print(f"   ↩️  Resuming from checkpoint at index {start_idx}")
            except Exception as e:
                print(f"   ⚠️  Could not load checkpoint: {e}")
        
        # Process MMS IDs
        total = len(mms_ids)
        for i, mms_id in enumerate(mms_ids[start_idx:], start=start_idx):
            exists, count, error = self.check_mms_exists(mms_id)
            
            result = {
                'F001': str(mms_id),  # Ensure it's a string
                'exists_in_alma': exists,
                'record_count': count,
                'error': error,
                'checked_at': datetime.now().isoformat()
            }
            results.append(result)
            
            # Show progress
            if (i + 1) % 10 == 0:
                exists_count = sum(1 for r in results if r['exists_in_alma'])
                print(f"   Progress: {i+1}/{total} - Found in Alma: {exists_count}/{i+1} ({exists_count/(i+1)*100:.1f}%)")
            
            # Save checkpoint
            if checkpoint_file and (i + 1) % CHECKPOINT_INTERVAL == 0:
                checkpoint = {
                    'last_index': i + 1,
                    'results': results,
                    'timestamp': datetime.now().isoformat()
                }
                with open(checkpoint_file, 'w') as f:
                    json.dump(checkpoint, f, indent=2)
                print(f"   💾 Checkpoint saved at index {i+1}")
        
        return results

print("\n🔍 Starting Alma SRU verification using reconstructed worklist...")

# Check if the reconstructed worklist exists
if not os.path.exists(worklist_path):
    print(f"❌ Reconstructed worklist not found: {worklist_path}")
    print("   Looking for alternative files...")
    
    # Try alternative locations
    alternative_files = [
        f"{output_dir}/harvard_api_worklist_reconstructed.csv",
        f"{output_dir}/harvard_api_worklist.csv",
        f"{output_dir}/physical_books_no_533_no_electronic_with_oclc.csv"
    ]
    
    for alt_file in alternative_files:
        if os.path.exists(alt_file):
            worklist_path = alt_file
            print(f"   ✅ Using: {alt_file}")
            break
    else:
        raise FileNotFoundError("No suitable worklist file found!")

# Load the worklist CSV file
print(f"\n📂 Loading reconstructed worklist from: {worklist_path}")
print(f"   File size: {os.path.getsize(worklist_path) / 1024 / 1024:.1f} MB")

# Read CSV file with proper dtype specification
df = pd.read_csv(worklist_path, dtype={'F001': str})

# Ensure F001 is string type to avoid merge conflicts
if 'F001' in df.columns:
    df['F001'] = df['F001'].astype(str)
    print(f"✅ Loaded {len(df):,} Penn-unique items from reconstructed worklist")
    print(f"   Columns available: {', '.join(df.columns[:10])}...")
    
    # This should be the CORRECT number (not 30M!)
    if len(df) > 10_000_000:
        print(f"\n⚠️ WARNING: Dataset seems too large ({len(df):,} records)")
        print("   This might be the exploded dataset, not the filtered unique records!")
        response = input("Continue anyway? (yes/no): ")
        if response.lower() != 'yes':
            raise ValueError("Aborted - dataset appears to be incorrect")
else:
    print("❌ F001 column not found in data!")
    raise ValueError("F001 column is required")

# Show sample of data to verify
print("\n📋 Sample of worklist data:")
print(df[['F001'] + [col for col in ['OCLC', 'ISBN', 'Title'] if col in df.columns]].head(3))

# Determine how many records to check
if RUN_MODE == "demo":
    df_to_check = df.head(100)
    print(f"\n🧪 DEMO MODE: Checking first 100 records only")
else:
    df_to_check = df
    print(f"\n🚀 FULL RUN: Checking all {len(df_to_check):,} records")
    print(f"   This is the CORRECT filtered dataset, not the 30M exploded records")

# Get MMS IDs to check (ensure they're strings)
mms_ids = df_to_check['F001'].astype(str).tolist()

# Initialize API checker
checker = AlmaAPIChecker()

# Set checkpoint file
checkpoint_file = f"{output_dir}/alma_check_checkpoint.json"

print(f"\n📡 Checking {len(mms_ids):,} MMS IDs in Alma...")
print(f"   Rate limit: {RATE_LIMIT_DELAY} seconds between calls")
print(f"   Estimated time: {len(mms_ids) * RATE_LIMIT_DELAY / 60:.1f} minutes")
print(f"   Estimated time: {len(mms_ids) * RATE_LIMIT_DELAY / 3600:.1f} hours")

# Check all MMS IDs
start_time = time.time()
results = checker.check_batch(mms_ids, checkpoint_file)
elapsed = time.time() - start_time

# Convert results to DataFrame with consistent string types
results_df = pd.DataFrame(results)
results_df['F001'] = results_df['F001'].astype(str)

# Ensure df_to_check also has F001 as string before merge
df_to_check['F001'] = df_to_check['F001'].astype(str)

# Merge with original data
df_with_alma = df_to_check.merge(
    results_df[['F001', 'exists_in_alma', 'record_count', 'error']],
    on='F001',
    how='left'
)

# Calculate statistics
total_checked = len(results_df)
exists_count = results_df['exists_in_alma'].sum()
missing_count = total_checked - exists_count
error_count = results_df['error'].notna().sum()

print(f"\n📊 Alma Verification Results:")
print(f"   Total checked: {total_checked:,}")
print(f"   Exists in Alma: {exists_count:,} ({exists_count/total_checked*100:.1f}%)")
print(f"   NOT in Alma: {missing_count:,} ({missing_count/total_checked*100:.1f}%)")
print(f"   Errors: {error_count:,}")
print(f"   Time taken: {elapsed/60:.1f} minutes ({elapsed/3600:.1f} hours)")

# Filter to only records that DO exist in Alma
df_alma_verified = df_with_alma[df_with_alma['exists_in_alma'] == True].copy()

print(f"\n✅ Records verified to exist in Alma: {len(df_alma_verified):,}")
print(f"   These are Penn-unique and EXIST in Alma")

# Check for HSP holdings
hsp_file = "/home/jovyan/work/July-2025-PODParquet/hsp_removed_mmsid.txt"
if os.path.exists(hsp_file):
    with open(hsp_file, 'r') as f:
        hsp_mms_ids = set(line.strip() for line in f if line.strip())
    
    df_alma_verified['is_hsp'] = df_alma_verified['F001'].isin(hsp_mms_ids)
    non_hsp = df_alma_verified[df_alma_verified['is_hsp'] == False]
    print(f"   After removing HSP holdings: {len(non_hsp):,}")
    df_final = non_hsp
else:
    df_final = df_alma_verified
    print("   (HSP check not applied - file not found)")

# Prepare data for Excel export
# Select relevant columns if they exist
export_cols = ['F001']
optional_cols = ['OCLC', 'ISBN', 'Title', 'F020', 'F035', 'F245', 'F010', 'F250', 'F260', 'F264', 
                 'material_category', 'match_key', 'oclc_number', 
                 'exists_in_alma', 'is_hsp']

for col in optional_cols:
    if col in df_final.columns:
        export_cols.append(col)

df_export = df_final[export_cols].copy()

# Add readable column names
rename_map = {
    'F001': 'MMS_ID',
    'F020': 'ISBN',
    'F035': 'System_Number',
    'F245': 'Title',
    'F010': 'LCCN',
    'F250': 'Edition',
    'F260': 'Publication',
    'F264': 'Publication_264',
    'oclc_number': 'OCLC'
}

df_export = df_export.rename(columns={k: v for k, v in rename_map.items() if k in df_export.columns})

# Save results
output_excel = f"{output_dir}/physical_books_alma_verified_from_worklist.xlsx"

with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
    # Full results with Alma status
    df_with_alma.to_excel(writer, sheet_name='All Checked', index=False)
    
    # Only Alma-verified records
    df_export.to_excel(writer, sheet_name='Alma Verified', index=False)
    
    # Summary statistics
    summary_df = pd.DataFrame({
        'Metric': [
            'Total items in worklist',
            'Checked in Alma',
            'Exists in Alma',
            'NOT in Alma',
            'Final verified count (after HSP removal)'
        ],
        'Count': [
            len(df),
            total_checked,
            exists_count,
            missing_count,
            len(df_final)
        ],
        'Percentage': [
            100.0,
            100.0,
            exists_count/total_checked*100 if total_checked > 0 else 0,
            missing_count/total_checked*100 if total_checked > 0 else 0,
            len(df_final)/len(df)*100
        ]
    })
    summary_df.to_excel(writer, sheet_name='Summary', index=False)
    
    # Records NOT in Alma (for investigation)
    df_missing = df_with_alma[df_with_alma['exists_in_alma'] == False]
    if len(df_missing) > 0:
        df_missing.to_excel(writer, sheet_name='Not In Alma', index=False)
        print(f"\n⚠️  {len(df_missing):,} records NOT found in Alma saved to 'Not In Alma' sheet")

print(f"\n💾 Results saved to: {output_excel}")
print(f"   Sheet 'Alma Verified' contains {len(df_final):,} records")
print(f"   Input was the reconstructed worklist (NOT the 30M exploded dataset)")

# Save results to JSON for record keeping
results_json = f"{output_dir}/alma_check_results_from_worklist.json"
with open(results_json, 'w') as f:
    json.dump({
        'check_date': datetime.now().isoformat(),
        'input_file': worklist_path,
        'total_in_worklist': len(df),
        'total_checked': total_checked,
        'exists_in_alma': int(exists_count),
        'not_in_alma': int(missing_count),
        'errors': int(error_count),
        'time_taken_minutes': elapsed/60,
        'run_mode': RUN_MODE
    }, f, indent=2)

print(f"📋 Check summary saved to: {results_json}")

# Clean up checkpoint file if successful
if os.path.exists(checkpoint_file):
    os.remove(checkpoint_file)
    print("🧹 Checkpoint file cleaned up")

print("\n✅ Alma verification complete!")
print(f"   Next step: Run BorrowDirect check on the {len(df_final):,} Alma-verified records")

In [ ]:
# First install selenium if not already installed
try:
    import selenium
    print("✅ Selenium is installed")
except ImportError:
    print("Installing selenium...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "selenium"])
    import selenium
    print("✅ Selenium installed successfully")

In [ ]:
# BorrowDirect Phase 1: Collect candidate BD record IDs (idempotent; safe to re-run)

print("\n" + "="*60)
print("BORROWDIRECT PHASE 1: COLLECT CANDIDATE RECORD IDS")
print("="*60)

import pandas as pd
import requests
import time
import json
import pickle
from datetime import datetime
from urllib.parse import quote
import re
import os
import builtins
from difflib import SequenceMatcher

# Configuration flags
RUN_MODE = "full"  # kept for compatibility; demo removed
CHECKPOINT_INTERVAL = 100  # Save progress every N records

alma_verified_excel = f"{output_dir}/physical_books_alma_verified_from_worklist.xlsx"

class BorrowDirectPhase1Collector:
    """Phase 1: Collect BorrowDirect record IDs for Penn items"""
    
    def __init__(self):
        self.base_url = "https://borrowdirect.reshare.indexdata.com/api/v1"
        self.search_url = f"{self.base_url}/search"
        self.rate_limit_delay = 0.5  # 2 requests per second
        self.session = requests.Session()
        self.session.headers.update({
            'Accept': 'application/json',
            'User-Agent': 'Penn-Library-Research/1.0'
        })
        self.last_request_time = 0
        
    def clean_isbn(self, isbn_field):
        if pd.isna(isbn_field) or not isbn_field:
            return None
        isbn_match = re.search(r'(\d{10,13})', str(isbn_field))
        if isbn_match:
            return isbn_match.group(1)
        return None
    
    def clean_title_for_search(self, title):
        if pd.isna(title) or not title:
            return None
        title = re.sub(r'\|[a-z]', ' ', str(title))
        title = re.sub(r'[^\w\s]', ' ', title)
        title = ' '.join(title.split())
        words = title.split()[:5]
        return ' '.join(words) if words else None
    
    def _rate_limit(self):
        current_time = time.time()
        time_since_last = current_time - self.last_request_time
        if time_since_last < self.rate_limit_delay:
            time.sleep(self.rate_limit_delay - time_since_last)
        self.last_request_time = time.time()
    
    def search_by_isbn(self, isbn):
        if not isbn:
            return []
        self._rate_limit()
        try:
            params = {
                'lookfor': isbn,
                'type': 'ISN',
                'field[]': ['id', 'title'],
                'limit': 20
            }
            response = self.session.get(self.search_url, params=params, timeout=30)
            if response.status_code == 200:
                data = response.json()
                bd_ids = []
                for record in data.get('records', []):
                    if 'id' in record:
                        bd_ids.append({
                            'bd_id': record['id'],
                            'bd_title': record.get('title', ''),
                            'search_method': 'isbn'
                        })
                return bd_ids
            else:
                return []
        except Exception as e:
            print(f"Error searching ISBN {isbn}: {e}")
            return []
    
    def search_by_title(self, title):
        clean_title = self.clean_title_for_search(title)
        if not clean_title:
            return []
        self._rate_limit()
        try:
            params = {
                'lookfor': clean_title,
                'type': 'AllFields',
                'field[]': ['id', 'title'],
                'limit': 20
            }
            response = self.session.get(self.search_url, params=params, timeout=30)
            if response.status_code == 200:
                data = response.json()
                bd_ids = []
                for record in data.get('records', []):
                    if 'id' in record:
                        bd_ids.append({
                            'bd_id': record['id'],
                            'bd_title': record.get('title', ''),
                            'search_method': 'title'
                        })
                return bd_ids
            else:
                return []
        except Exception as e:
            print(f"Error searching title: {e}")
            return []
    
    def collect_candidates(self, record):
        candidates = []
        isbn = self.clean_isbn(record.get('ISBN'))
        candidates += self.search_by_isbn(isbn)
        title = record.get('Title')
        title_cands = self.search_by_title(title)
        existing_ids = {c['bd_id'] for c in candidates}
        for tc in title_cands:
            if tc['bd_id'] not in existing_ids:
                candidates.append(tc)
        return candidates

print("\n🔍 Phase 1: Collecting BorrowDirect candidate record IDs...")

if not os.path.exists(alma_verified_excel):
    print(f"❌ Required Excel not found: {alma_verified_excel}\n   Please run Alma verification earlier in the notebook.")
else:
    # Load Alma-verified Excel as authoritative input
    original_df = pd.read_excel(
        alma_verified_excel,
        sheet_name='Alma Verified',
        dtype={'OCLC': str, 'ISBN': str, 'F001': str}
    )

    # Idempotent merge from any prior Phase 1 results
    phase1_file = f"{output_dir}/bd_phase1_results_full.pkl"
    merged_file = f"{output_dir}/bd_phase1_merged_results.pkl"  # optional legacy

    existing = {}
    if os.path.exists(merged_file):
        try:
            with open(merged_file, 'rb') as f:
                payload = pickle.load(f)
                for r in payload.get('results', []):
                    existing[r['F001']] = r
            print(f"📦 Existing Phase 1 merged results: {len(existing):,} records")
        except Exception as e:
            print("  ! Failed to read merged phase 1 results:", e)

    results = [] if not existing else list(existing.values())

    if 'F001' not in original_df.columns:
        print("❌ Input is missing F001 column.")
    else:
        base_ids = original_df['F001'].astype(str).tolist()
        pending_ids = [mid for mid in base_ids if mid not in existing]
        print(f"  - Candidate MMS IDs: {len(base_ids):,}")
        print(f"  - Already collected: {len(existing):,}")
        print(f"  - Remaining to collect: {len(pending_ids):,}")

        collector = BorrowDirectPhase1Collector()
        checkpoint_file = f"{output_dir}/bd_phase1_checkpoint.pkl"
        start_idx = 0
        if os.path.exists(checkpoint_file):
            try:
                with open(checkpoint_file, 'rb') as f:
                    cp = pickle.load(f)
                    start_idx = cp.get('last_index', 0)
                    prev = cp.get('results', [])
                    results_map = {r['F001']: r for r in results}
                    for r in prev:
                        results_map[r['F001']] = r
                    results = list(results_map.values())
                print(f"  ↩️  Resuming Phase 1 from checkpoint index {start_idx}")
            except Exception as e:
                print("  ! Failed to load Phase 1 checkpoint:", e)

        found_count = 0
        for i, mms_id in enumerate(pending_ids[start_idx:], start=start_idx):
            rec = original_df.loc[original_df['F001'] == mms_id].iloc[0].to_dict()
            cands = collector.collect_candidates(rec)
            result = {
                'F001': mms_id,
                'num_candidates': len(cands),
                'has_candidates': len(cands) > 0,
                'candidates': cands,
                'search_title': rec.get('Title')
            }
            results.append(result)
            if cands:
                found_count += 1

            if (i + 1) % CHECKPOINT_INTERVAL == 0:
                try:
                    with open(checkpoint_file, 'wb') as f:
                        pickle.dump({'last_index': i + 1, 'results': results}, f)
                    print(f"   💾 Phase 1 checkpoint at {i + 1}")
                except Exception as e:
                    print("   ! Failed to save Phase 1 checkpoint:", e)

        print(f"\n📊 Phase 1 collection completed for {len(pending_ids):,} new MMS IDs")
        records_with_candidates = sum(1 for r in results if r.get('num_candidates', 0) > 0)
        print(f"  - Records with BD candidates: {records_with_candidates} ({records_with_candidates/len(results)*100:.1f}%)")

        with open(phase1_file, 'wb') as f:
            pickle.dump({
                'results': results,
                'original_df': original_df,
                'run_mode': RUN_MODE,
                'timestamp': datetime.now().isoformat(),
            }, f)
        print(f"  💾 Saved Phase 1 results: {phase1_file}")

In [ ]:
# BorrowDirect Phase 2: Full Run with Selenium (delta-aware + inline Excel build)

print("\n" + "="*60)
print("BORROWDIRECT PHASE 2: FULL RUN WITH SELENIUM — DELTA + APPEND")
print("="*60)

import os
import pickle
import time
from datetime import datetime
import logging
from collections import deque
import threading
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from difflib import SequenceMatcher
import html
import unicodedata
import pandas as pd

# NOTE: Class SeleniumBorrowDirectParser is defined in the previous Phase 2 cell and reused here.

print("\n🔎 Starting BorrowDirect Phase 2 in delta/append mode...")
print("📌 Behavior: processes only MMS IDs not already present in the merged Phase 2 results; then rebuilds the validated Excel.")

# Inputs (prefer merged; fall back to full)
phase1_file_merged = f"{output_dir}/bd_phase1_merged_results.pkl"
phase1_file_full = f"{output_dir}/bd_phase1_results_full.pkl"

# Canonical/cached outputs
phase2_merged_file = f"{output_dir}/bd_phase2_merged_results.pkl"
checkpoint_file = f"{output_dir}/bd_phase2_checkpoint.pkl"

# Downstream Excel paths
alma_verified_excel = f"{output_dir}/physical_books_NOT_harvard_alma_verified.xlsx"
bd_validated_excel = f"{output_dir}/penn_unique_borrowdirect_validated.xlsx"

# Resolve Phase 1 input
phase1_path = None
if os.path.exists(phase1_file_merged):
    phase1_path = phase1_file_merged
elif os.path.exists(phase1_file_full):
    phase1_path = phase1_file_full

if not phase1_path:
    print(f"❌ Phase 1 results not found: {phase1_file_merged} or {phase1_file_full}")
else:
    # Load Phase 1 records
    with open(phase1_path, 'rb') as f:
        phase1_data = pickle.load(f)
    records = phase1_data.get('results', [])
    records_with_candidates = [r for r in records if r.get('candidates', [])]

    print(f"📊 Loaded {len(records):,} records from Phase 1; with candidates: {len(records_with_candidates):,}")

    # Load existing merged Phase 2 results if available
    existing_map = {}
    existing_results = []
    if os.path.exists(phase2_merged_file):
        try:
            with open(phase2_merged_file, 'rb') as f:
                phase2_data = pickle.load(f)
                existing_results = phase2_data.get('results', [])
                existing_map = {r['F001']: r for r in existing_results}
            print(f"📦 Existing Phase 2 merged results found for {len(existing_map):,} MMS IDs")
        except Exception as e:
            print("⚠️ Failed to read existing Phase 2 merged results:", e)

    # Build delta list
    pending = [r for r in records_with_candidates if r['F001'] not in existing_map]
    print(f"  - Pending to process: {len(pending):,}")

    # Resume from checkpoint if available (applies to delta list)
    start_idx = 0
    new_results = []
    successful_parses = 0
    if os.path.exists(checkpoint_file):
        try:
            print("📂 Found checkpoint; resuming...")
            with open(checkpoint_file, 'rb') as f:
                cp = pickle.load(f)
                start_idx = cp.get('last_index', 0)
                new_results = cp.get('results', [])
                successful_parses = cp.get('successful_parses', 0)
            print(f"  ↩️ Resume index: {start_idx}")
        except Exception as e:
            print("⚠️ Failed to load checkpoint:", e)

    # Configuration
    RUN_MODE = "delta"  # process only pending
    BATCH_SIZE = 100

    # If there's work to do, run Selenium parser
    if pending:
        parser = SeleniumBorrowDirectParser(headless=True)
        try:
            start_time = time.time()
            total = len(pending)
            for batch_start in range(start_idx, total, BATCH_SIZE):
                batch_end = min(batch_start + BATCH_SIZE, total)
                batch = pending[batch_start:batch_end]

                print(f"\n📦 Processing batch {batch_start}-{batch_end} of {total}")
                batch_results, batch_parses = parser.process_batch(batch)
                new_results.extend(batch_results)
                successful_parses += batch_parses

                # Save checkpoint for delta progress
                cp = {
                    'last_index': batch_end,
                    'results': new_results,
                    'successful_parses': successful_parses,
                    'timestamp': datetime.now().isoformat(),
                }
                with open(checkpoint_file, 'wb') as f:
                    pickle.dump(cp, f)
                print(f"  💾 Checkpoint saved at {batch_end}")

            elapsed = time.time() - start_time
            print("\n" + "="*60)
            print("PHASE 2 DELTA PROCESSING COMPLETE")
            print("="*60)
            print(f"⏱️ Processed {len(new_results)} new records in {elapsed/60:.1f} minutes; parses: {successful_parses}")
        finally:
            parser.cleanup()

        # Merge existing + new results keyed by F001
        merged_map = {r['F001']: r for r in existing_results}
        for r in new_results:
            merged_map[r['F001']] = r
        merged_results = list(merged_map.values())

        # Save merged canonical results
        with open(phase2_merged_file, 'wb') as f:
            pickle.dump({
                'results': merged_results,
                'timestamp': datetime.now().isoformat(),
                'run_mode': RUN_MODE,
                'successful_parses_total': successful_parses,
            }, f)
        print(f"✅ Merged Phase 2 results saved: {phase2_merged_file} ({len(merged_results):,} records)")

        # Clean up checkpoint on success
        if os.path.exists(checkpoint_file):
            os.remove(checkpoint_file)
            print("🧹 Removed checkpoint file")
    else:
        print("✅ No pending MMS IDs to process; using existing Phase 2 merged results")
        merged_results = existing_results

    # Always (re)build the BorrowDirect validated Excel from merged results
    if not os.path.exists(alma_verified_excel):
        print(f"❌ Original Excel not found: {alma_verified_excel}\n   Run the Alma verification cell first.")
    else:
        try:
            df_original = pd.read_excel(
                alma_verified_excel,
                sheet_name='Alma Verified',
                dtype={'OCLC': str, 'ISBN': str, 'F001': str}
            )
            # Build summary from merged Phase 2
            bd_rows = []
            for res in merged_results:
                bd_rows.append({
                    'F001': res['F001'],
                    'found_in_bd': bool(res.get('found_in_bd', False)),
                    'is_penn_only': bool(res.get('is_penn_only', False)),
                    'num_candidates_checked': int(res.get('num_candidates_checked', 0)),
                })
            df_bd = pd.DataFrame(bd_rows)

            df_final = df_original.merge(df_bd, on='F001', how='left')
            df_final['found_in_bd'] = df_final['found_in_bd'].fillna(False)
            df_final['is_penn_only'] = df_final['is_penn_only'].fillna(False)
            df_final['num_candidates_checked'] = df_final['num_candidates_checked'].fillna(0)

            total_records = len(df_final)
            found_in_bd = int(df_final['found_in_bd'].sum())
            penn_only = int(df_final['is_penn_only'].sum())
            not_found = total_records - found_in_bd
            available_elsewhere = found_in_bd - penn_only

            print("\n📊 BorrowDirect Validation Results (from merged Phase 2):")
            print(f"  - Total records: {total_records:,}")
            print(f"  - Found in BorrowDirect: {found_in_bd:,} ({found_in_bd/total_records*100:.1f}%)")
            print(f"    • Penn-only: {penn_only:,}")
            print(f"    • Available elsewhere: {available_elsewhere:,}")
            print(f"  - NOT in BorrowDirect: {not_found:,} ({not_found/total_records*100:.1f}%)")

            with pd.ExcelWriter(bd_validated_excel, engine='openpyxl') as writer:
                df_final.to_excel(writer, sheet_name='BD Validated', index=False)
                summary_df = pd.DataFrame({
                    'Metric': [
                        'Total Records',
                        'Found in BorrowDirect',
                        'Penn-only in BD',
                        'Available Elsewhere',
                        'Not in BorrowDirect',
                    ],
                    'Count': [
                        total_records,
                        found_in_bd,
                        penn_only,
                        available_elsewhere,
                        not_found,
                    ],
                    'Percentage': [
                        100.0,
                        found_in_bd/total_records*100,
                        penn_only/total_records*100,
                        available_elsewhere/total_records*100,
                        not_found/total_records*100,
                    ],
                })
                summary_df.to_excel(writer, sheet_name='Summary', index=False)

            print(f"\n💾 BorrowDirect validated Excel saved: {bd_validated_excel}")
            print("   This file is ready for the HathiTrust filtering step.")
        except Exception as e:
            print("⚠️ Failed to build BD validated Excel:", e)

print("\n✅ Phase 2 delta+append complete.")

# HathiTrust Check

In [ ]:
print("\n" + "="*60)
print("FILTERING BORROWDIRECT RESULTS FOR HATHITRUST CHECK")
print("="*60)

import os
import pandas as pd
from datetime import datetime

# Ensure output_dir is available and valid
if 'output_dir' not in globals():
    output_dir = os.path.join(os.getcwd(), 'pod-processing-outputs')
    os.makedirs(output_dir, exist_ok=True)

# Load the BorrowDirect validated results
bd_results_path = f"{output_dir}/penn_unique_borrowdirect_validated.xlsx"

if os.path.exists(bd_results_path):
    # Load the BorrowDirect validated data
    df_bd = pd.read_excel(
        bd_results_path,
        sheet_name='BD Validated',
        dtype={'OCLC': str, 'F001': str, 'ISBN': str}
    )

    print(f"✅ Loaded {len(df_bd):,} BorrowDirect validated records")

    # Filter to truly Penn-unique items:
    # - Not found in BorrowDirect OR
    # - Found but Penn-only in BorrowDirect
    if 'found_in_bd' in df_bd.columns and 'is_penn_only' in df_bd.columns:
        df_penn_unique = df_bd[
            (~df_bd['found_in_bd']) |
            (df_bd['is_penn_only'])
        ]

        print(f"\n? Filtering results:")
        print(f"  - Records before filtering: {len(df_bd):,}")
        print(f"  - Records after filtering (truly Penn-unique): {len(df_penn_unique):,}")
        print(f"  - Removed {len(df_bd) - len(df_penn_unique):,} records available at other institutions")

        # Optional: breakdown
        if len(df_penn_unique) > 0:
            not_found_count = int((~df_penn_unique['found_in_bd']).sum())
            penn_only_count = int(df_penn_unique['is_penn_only'].sum())
            print(f"\n?️ Breakdown of Penn-unique items:")
            print(f"  - Not found in BorrowDirect: {not_found_count:,}")
            print(f"  - Found but Penn-only: {penn_only_count:,}")

        # Save to a new Excel file for HathiTrust checking
        hathitrust_input_path = f"{output_dir}/physical_books_for_hathitrust_check.xlsx"
        df_penn_unique.to_excel(hathitrust_input_path, sheet_name='Penn Unique', index=False)

        print(f"\n💾 Saved filtered data for HathiTrust checking: {hathitrust_input_path}")
        print("   This file contains only items truly unique to Penn.")
    else:
        print("⚠️ Required columns 'found_in_bd' or 'is_penn_only' not found in the dataset.")
        print("Please ensure the BorrowDirect Phase 2 cell completed successfully.")
else:
    print(f"❌ BorrowDirect results not found: {bd_results_path}")
    print("Please run the BorrowDirect Phase 2 cell first.")

In [ ]:
# HathiTrust check Penn-unique items (Alma verified)
print("\n" + "="*60)
print("HATHITRUST CHECK FOR PENN-UNIQUE ITEMS NOT AT HARVARD (ALMA VERIFIED)")
print("="*60)

import sys
import os
import pandas as pd
import json
import time
import random
import pickle
import requests
from datetime import datetime
import numpy as np

# Graceful tqdm import (avoid pip install in notebook)
try:
    from tqdm.auto import tqdm
except Exception:
    def tqdm(x, **kwargs):
        return x

# Ensure output_dir exists and is consistent (no hard-coded Linux path)
if 'output_dir' not in globals():
    output_dir = os.path.join(os.getcwd(), 'pod-processing-outputs')
    os.makedirs(output_dir, exist_ok=True)

# Ensure hathitrust module path is resolvable from repo-relative directory
repo_root = os.path.abspath(os.path.join(os.getcwd()))
hathitrust_dir = os.path.join(repo_root, 'hathitrust')
os.makedirs(hathitrust_dir, exist_ok=True)
if hathitrust_dir not in sys.path:
    sys.path.insert(0, hathitrust_dir)

try:
    # Import HathiTrust scanner
    from hathitrust_availability_checker_excel import HathiTrustFullScanner

    # Load the filtered BorrowDirect results file created in the prior cell
    print("\n📂 Loading Penn-unique items for HathiTrust check...")
    excel_file = f"{output_dir}/physical_books_for_hathitrust_check.xlsx"
    sheet_name = 'Penn Unique'

    if os.path.exists(excel_file):
        df_penn_unique = pd.read_excel(
            excel_file,
            sheet_name=sheet_name,
            dtype={'OCLC': str, 'F001': str}
        )
        print(f"✅ Loaded {len(df_penn_unique):,} Penn-unique items for HathiTrust check")
        print("   These records are:")
        print("   ✓ Unique to Penn")
        print("   ✓ NOT at Harvard")
        print("   ✓ Exist in Alma")
        print("   ✓ NOT HSP holdings")
        print("   ✓ Either NOT in BorrowDirect or Penn-only in BorrowDirect")

        # Clean OCLC numbers
        if 'OCLC' in df_penn_unique.columns:
            df_penn_unique['OCLC'] = df_penn_unique['OCLC'].fillna('')
            df_penn_unique['OCLC'] = df_penn_unique['OCLC'].astype(str).str.replace('.0', '', regex=False)
            df_penn_unique['OCLC'] = df_penn_unique['OCLC'].replace('nan', '')
            oclc_present = (df_penn_unique['OCLC'] != '').sum()
            print(f"   OCLC numbers present: {oclc_present:,} ({oclc_present/len(df_penn_unique)*100:.1f}%)")

        print(f"\n🗂️ Available columns: {list(df_penn_unique.columns)}")
        print("\n🧪 Sample records:")
        sample_cols = ['F001', 'OCLC', 'ISBN', 'Title']
        available_cols = [col for col in sample_cols if col in df_penn_unique.columns]
        print(df_penn_unique[available_cols].head(3).to_string(index=False))

    else:
        print(f"❌ Excel file not found: {excel_file}")
        print("Please run the BorrowDirect filtering cell to generate this file.")
        raise FileNotFoundError(f"Missing file: {excel_file}")

    # Check all unique items in HathiTrust
    print(f"\n✅ Will check all {len(df_penn_unique):,} items in HathiTrust")
    print("   These are high priority: Penn-unique, not at Harvard, in Alma, no HSP, and not broadly available via BorrowDirect")

    print(f"\n🔄 Preparing records for HathiTrust availability check...")
    dataset_name = f"penn_unique_not_harvard_alma_{len(df_penn_unique)}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    temp_file = f"{output_dir}/temp_hathitrust_{dataset_name}.xlsx"

    hathi_df = pd.DataFrame({
        'MMS_ID': df_penn_unique['F001'].astype(str),
        'OCLC': df_penn_unique['OCLC'].astype(str) if 'OCLC' in df_penn_unique.columns else '',
        'F245': df_penn_unique['Title'] if 'Title' in df_penn_unique.columns else '',
        'F020_str': df_penn_unique['ISBN'].astype(str) if 'ISBN' in df_penn_unique.columns else '',
        'F010_str': df_penn_unique['LCCN'].astype(str) if 'LCCN' in df_penn_unique.columns else '',
        'F260_str': df_penn_unique['Publication'].astype(str) if 'Publication' in df_penn_unique.columns else '',
        'material_category': df_penn_unique['material_category'] if 'material_category' in df_penn_unique.columns else '',
        'match_key': df_penn_unique['match_key'] if 'match_key' in df_penn_unique.columns else '',
    }).fillna('')

    hathi_df['OCLC'] = hathi_df['OCLC'].str.replace('nan', '').str.replace('.0', '')

    hathi_df.to_excel(temp_file, index=False)
    print(f"✅ Prepared {len(hathi_df):,} records for HathiTrust check")
    oclc_count = int((hathi_df['OCLC'] != '').sum())
    print(f"   Records with OCLC for HathiTrust: {oclc_count:,} ({oclc_count/len(hathi_df)*100:.1f}%)")

    reports_base = f"{output_dir}/hathitrust_reports"
    os.makedirs(reports_base, exist_ok=True)
    output_report_dir = f"{reports_base}/{dataset_name}"
    os.makedirs(output_report_dir, exist_ok=True)

    print("\n🔎 Initializing HathiTrust scanner...")
    from hathitrust_availability_checker_excel import HathiTrustFullScanner
    scanner = HathiTrustFullScanner(
        rate_limit_delay=0.5,
        max_workers=2,
    )

    print(f"\n🚀 Starting HathiTrust availability check...")
    print("   ✓ Penn-unique (conservative)\n   ✓ NOT at Harvard (title-verified)\n   ✓ In Alma\n   ✓ NOT HSP\n   ✓ Not in BD or Penn-only in BD")
    estimated_time = (len(hathi_df) * 0.5) / 60
    print(f"\n⏱️ Estimated time: {estimated_time:.1f} minutes ({estimated_time/60:.1f} hours)")

    scanner.scan_full_file(temp_file, start_from=0, batch_size=50)

    print(f"\n✅ HathiTrust check complete!")

    if os.path.exists(temp_file):
        os.remove(temp_file)
        print("   Cleaned up temporary file")

    result_files = [
        os.path.join(output_report_dir, f)
        for f in os.listdir(output_report_dir)
        if f.startswith('hathitrust_scan_results_') and f.endswith('.csv')
    ]

    if result_files:
        results_df = pd.read_csv(result_files[0])
        total_checked = len(results_df)
        found_in_hathi = int(results_df['found'].sum()) if 'found' in results_df.columns else 0
        not_in_hathi = total_checked - found_in_hathi

        print(f"\n📊 RESULTS SUMMARY:")
        print(f"   Total checked: {total_checked:,}")
        print(f"   Found in HathiTrust: {found_in_hathi:,} ({found_in_hathi/total_checked*100:.1f}%)")
        print(f"   NOT in HathiTrust: {not_in_hathi:,} ({not_in_hathi/total_checked*100:.1f}%)")

        if 'access_type' in results_df.columns:
            access_counts = results_df['access_type'].value_counts()
            print("\n🗂️ Access levels for items in HathiTrust:")
            for access_type, count in access_counts.items():
                print(f"   - {access_type}: {count:,}")

    summary_info = {
        'check_date': datetime.now().isoformat(),
        'dataset': 'penn_unique_not_at_harvard_alma_verified_bd_filtered',
        'total_items_checked': len(df_penn_unique),
        'items_with_oclc': int(oclc_count),
        'oclc_coverage_percent': float(oclc_count/len(hathi_df)*100),
        'description': 'Penn-unique, not at Harvard, in Alma, no HSP holdings, filtered by BorrowDirect',
        'priority': 'HIGHEST',
        'source_file': excel_file,
        'filtering_applied': [
            'Conservative unique filtering (standard identifiers)',
            'ISBN deduplication',
            'F533 reproductions removed',
            'HSP records removed',
            'RDA electronic resources removed',
            'Harvard API check with title verification',
            'Alma existence verification',
            'HSP holdings exclusion',
            'BorrowDirect availability filtering',
        ],
        'results_location': output_report_dir,
    }

    summary_file = f"{output_dir}/hathitrust_penn_not_harvard_alma_{len(df_penn_unique)}_summary.json"
    with open(summary_file, "w") as f:
        json.dump(summary_info, f, indent=2)
    print(f"\n💾 Summary info saved to: {summary_file}")
    print(f"\n📁 RESULTS LOCATION:\n   {output_report_dir}")

except ImportError as e:
    print(f"❌ Could not import HathiTrust scanner: {e}")
    print("Please ensure hathitrust_availability_checker_excel.py is in the hathitrust/ directory relative to the repo root.")
except Exception as e:
    print(f"❌ Error during HathiTrust check: {str(e)}")
    import traceback
    traceback.print_exc()
    if 'temp_file' in locals() and os.path.exists(temp_file):
        os.remove(temp_file)
        print(f"   Cleaned up temporary file: {temp_file}")

In [1]:
# Generate Final Excel Report with HathiTrust Results
print("\n" + "="*60)
print("GENERATING FINAL EXCEL REPORT WITH HATHITRUST RESULTS")
print("="*60)

import os
import pandas as pd
from datetime import datetime
import glob

# Define the output directory
output_dir = "/home/jovyan/work/July-2025-PODParquet/pod-processing-outputs"

try:
    # 1. Load the Penn-unique records (filtered for HathiTrust check)
    print("\n📂 Loading Penn-unique records...")
    hathitrust_input = f"{output_dir}/physical_books_for_hathitrust_check.xlsx"
    
    if not os.path.exists(hathitrust_input):
        raise FileNotFoundError(f"Missing file: {hathitrust_input}")
    
    df_penn_unique = pd.read_excel(
        hathitrust_input,
        sheet_name='Penn Unique',
        dtype={'OCLC': str, 'F001': str, 'ISBN': str}
    )
    print(f"✅ Loaded {len(df_penn_unique):,} Penn-unique records")
    
    # 2. Find the latest HathiTrust results
    print("\n📂 Looking for HathiTrust scan results...")
    reports_base = f"{output_dir}/hathitrust_reports"
    
    # Find all scan directories (sorted by date, most recent first)
    scan_dirs = sorted(
        [d for d in glob.glob(f"{reports_base}/penn_unique_*") if os.path.isdir(d)],
        key=os.path.getmtime,
        reverse=True
    )
    
    hathi_results_df = None
    latest_scan_dir = None
    
    if scan_dirs:
        latest_scan_dir = scan_dirs[0]
        print(f"✅ Found scan directory: {os.path.basename(latest_scan_dir)}")
        
        # Look for results CSV in the directory
        result_files = glob.glob(f"{latest_scan_dir}/hathitrust_scan_results_*.csv")
        
        if result_files:
            latest_results = sorted(result_files, key=os.path.getmtime, reverse=True)[0]
            hathi_results_df = pd.read_csv(latest_results, dtype={'MMS_ID': str, 'OCLC': str})
            print(f"✅ Loaded HathiTrust results: {len(hathi_results_df):,} records")
            print(f"   From: {os.path.basename(latest_results)}")
        else:
            print("⚠️ No results CSV found in scan directory")
    else:
        print("⚠️ No HathiTrust scan results found")
        print("   Please run the HathiTrust check cell first")
    
    # 3. Merge HathiTrust results with Penn-unique data
    print("\n🔄 Merging HathiTrust results with Penn-unique data...")
    
    if hathi_results_df is not None:
        # Clean MMS_ID columns for matching
        df_penn_unique['F001_clean'] = df_penn_unique['F001'].astype(str).str.strip()
        hathi_results_df['MMS_ID_clean'] = hathi_results_df['MMS_ID'].astype(str).str.strip()
        
        # Merge on MMS_ID
        df_final = df_penn_unique.merge(
            hathi_results_df[['MMS_ID_clean', 'found', 'num_items', 'access_type', 'match_type']],
            left_on='F001_clean',
            right_on='MMS_ID_clean',
            how='left',
            suffixes=('', '_hathi')
        )
        
        # Drop temporary columns
        df_final = df_final.drop(columns=['F001_clean', 'MMS_ID_clean'], errors='ignore')
        
        # Rename HathiTrust columns for clarity
        df_final = df_final.rename(columns={
            'found': 'in_hathitrust',
            'num_items': 'hathi_item_count',
            'access_type': 'hathi_access',
            'match_type': 'hathi_match_type'
        })
        
        print(f"✅ Merged data: {len(df_final):,} records")
        
        # Calculate statistics
        total = len(df_final)
        in_hathi = df_final['in_hathitrust'].sum() if 'in_hathitrust' in df_final.columns else 0
        not_in_hathi = total - in_hathi
        
        print(f"\n📊 HathiTrust Coverage:")
        print(f"   Total Penn-unique items: {total:,}")
        print(f"   Found in HathiTrust: {in_hathi:,} ({in_hathi/total*100:.1f}%)")
        print(f"   NOT in HathiTrust: {not_in_hathi:,} ({not_in_hathi/total*100:.1f}%)")
        
        if 'hathi_access' in df_final.columns:
            print(f"\n📋 Access levels for items in HathiTrust:")
            access_counts = df_final['hathi_access'].value_counts()
            for access_type, count in access_counts.items():
                if pd.notna(access_type):
                    print(f"   - {access_type}: {count:,}")
        
    else:
        df_final = df_penn_unique.copy()
        print("⚠️ No HathiTrust results to merge - using Penn-unique data only")
    
    # 4. Create priority classification
    print("\n🎯 Classifying  priorities...")
    
    def classify_priority(row):
        """Classify priority based on HathiTrust status"""
        if 'in_hathitrust' not in row or pd.isna(row.get('in_hathitrust')):
            return 'PRIORITY 1: Unknown (HathiTrust not checked)'
        
        if not row['in_hathitrust']:
            return 'PRIORITY 1: Not in HathiTrust (HIGHEST)'
        
        access = row.get('hathi_access', '')
        if pd.isna(access) or access == '':
            return 'PRIORITY 2: In HathiTrust (access unknown)'
        
        if 'limited' in str(access).lower() or 'restricted' in str(access).lower():
            return 'PRIORITY 2: In HathiTrust (restricted access)'
        
        if 'full' in str(access).lower():
            return 'PRIORITY 3: In HathiTrust (full access available)'
        
        return 'PRIORITY 2: In HathiTrust (access unknown)'
    
    df_final['preservation_priority'] = df_final.apply(classify_priority, axis=1)
    
    # Show priority breakdown
    print(f"\n📋 Priority Breakdown:")
    priority_counts = df_final['preservation_priority'].value_counts().sort_index()
    for priority, count in priority_counts.items():
        print(f"   {priority}: {count:,}")
    
    # 5. Generate final Excel report
    print("\n📄 Generating final Excel report...")
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    final_report_path = f"{output_dir}/FINAL_penn_preservation_priorities_{timestamp}.xlsx"
    
    # Reorder columns for better readability
    priority_cols = ['preservation_priority', 'in_hathitrust', 'hathi_access', 'hathi_item_count']
    identity_cols = ['F001', 'OCLC', 'ISBN', 'LCCN', 'Title', 'Author', 'Publication']
    bd_cols = ['found_in_bd', 'is_penn_only', 'bd_num_holdings', 'bd_holding_institutions']
    other_cols = [col for col in df_final.columns if col not in priority_cols + identity_cols + bd_cols]
    
    # Build ordered column list (only include columns that exist)
    ordered_cols = []
    for col in priority_cols + identity_cols + bd_cols + other_cols:
        if col in df_final.columns:
            ordered_cols.append(col)
    
    df_final = df_final[ordered_cols]
    
    # Create Excel writer with multiple sheets
    with pd.ExcelWriter(final_report_path, engine='openpyxl') as writer:
        # Main sheet: All records sorted by priority
        df_final.sort_values('preservation_priority').to_excel(
            writer, 
            sheet_name='All Records', 
            index=False
        )
        
        # Priority 1: Items NOT in HathiTrust (highest priority)
        priority1 = df_final[df_final['preservation_priority'].str.contains('PRIORITY 1', na=False)]
        if len(priority1) > 0:
            priority1.to_excel(writer, sheet_name='Priority 1 - Not in HT', index=False)
        
        # Priority 2: Items in HathiTrust but restricted
        priority2 = df_final[df_final['preservation_priority'].str.contains('PRIORITY 2', na=False)]
        if len(priority2) > 0:
            priority2.to_excel(writer, sheet_name='Priority 2 - Restricted', index=False)
        
        # Priority 3: Items in HathiTrust with full access
        priority3 = df_final[df_final['preservation_priority'].str.contains('PRIORITY 3', na=False)]
        if len(priority3) > 0:
            priority3.to_excel(writer, sheet_name='Priority 3 - Full Access', index=False)
        
        # Summary statistics sheet
        summary_data = {
            'Metric': [
                'Report Generated',
                'Total Penn-Unique Items',
                'Items Checked in HathiTrust',
                'Found in HathiTrust',
                'NOT in HathiTrust',
                'HathiTrust Coverage %',
                'Priority 1 (Not in HT)',
                'Priority 2 (Restricted)',
                'Priority 3 (Full Access)',
                '',
                'Filtering Applied',
                '- Conservative unique filtering',
                '- Harvard verification',
                '- Alma validation',
                '- HSP exclusion',
                '- BorrowDirect filtering',
                '- HathiTrust check'
            ],
            'Value': [
                datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                len(df_final),
                len(df_final) if hathi_results_df is not None else 0,
                int(in_hathi) if hathi_results_df is not None else 0,
                int(not_in_hathi) if hathi_results_df is not None else 0,
                f"{in_hathi/total*100:.1f}%" if hathi_results_df is not None else 'N/A',
                len(priority1),
                len(priority2),
                len(priority3),
                '',
                '',
                '✓',
                '✓',
                '✓',
                '✓',
                '✓',
                '✓' if hathi_results_df is not None else 'Pending'
            ]
        }
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
    
    print(f"\n✅ Final report generated successfully!")
    print(f"📁 Location: {final_report_path}")
    print(f"\n📊 Report contains {len(df_final):,} records across multiple sheets:")
    print(f"   ✓ 'All Records' - Complete dataset sorted by priority")
    print(f"   ✓ 'Priority 1 - Not in HT' - {len(priority1):,} records (HIGHEST PRIORITY)")
    print(f"   ✓ 'Priority 2 - Restricted' - {len(priority2):,} records")
    print(f"   ✓ 'Priority 3 - Full Access' - {len(priority3):,} records")
    print(f"   ✓ 'Summary' - Key statistics")
    
    print("\n🎯 NEXT STEPS:")
    print("   1. Review Priority 1 items - these are NOT in HathiTrust!")
    print("   2. Review Priority 2 items - these have restricted access in HathiTrust")
    print("   3. Priority 3 items are already fully accessible in HathiTrust")
    
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("Please ensure all previous cells have been run successfully:")
    print("   1. BorrowDirect Phase 1 (full collection)")
    print("   2. BorrowDirect Phase 1.5 (enhanced search)")
    print("   3. Merge Phase 1 + 1.5")
    print("   4. BorrowDirect Phase 2 (Selenium verification)")
    print("   5. Filter for HathiTrust check")
    print("   6. HathiTrust availability check")
    
except Exception as e:
    print(f"❌ Error generating report: {str(e)}")
    import traceback
    traceback.print_exc()

In [ ]:
# Optional Cleanup - Run this to free memory after analysis
def cleanup_spark_resources():
    """Clean up all cached DataFrames and temporary views"""
    try:
        # Get all cached DataFrames
        cached_count = len(spark.sparkContext._jsc.getPersistentRDDs().items())
        
        for (id, rdd) in spark.sparkContext._jsc.getPersistentRDDs().items():
            rdd.unpersist()
        
        # Drop all temporary views
        temp_views = [view.name for view in spark.catalog.listTables() if view.isTemporary]
        for view_name in temp_views:
            spark.catalog.dropTempView(view_name)
        
        print(f"✅ Cleaned up {cached_count} cached DataFrames and {len(temp_views)} temporary views")
        print("💡 Memory freed. You can safely re-run the notebook or close it.")
    except Exception as e:
        print(f"⚠️ Cleanup warning: {e}")

# Run cleanup
cleanup_spark_resources()

# Optional: Show memory status
print("\n📊 Spark UI still available at:", spark.sparkContext.uiWebUrl)
print("Check the Storage tab to verify all caches are cleared")